# Link al libro que estoy usando para armar el LLM: [Build a Large Language Model from Scratch - Sebastian Raschka](https://drive.google.com/file/d/1WkucQZgK4RENhGG_lm75yRqcQWw-Pxzk/view?usp=sharing)

# **Capítulo 1:** El capítulo 1 se encuentra el link al pdf dispinible para descarga en el link anterior. Es puramente introductorio les recomiendo leer el primer capítulo.

# **Capitulo 2:** [Celdas de código mas comentarios explicativos.](#scrollTo=9l55Q2tNWDdU)

# **Capitulo 3:** [Celdas de código mas comentarios explicativos.](#scrollTo=LQEg0nPD6-ux)


In [1]:
with open('./sample_data/llm_from_scratch/the-verdict.txt', 'r', encoding='utf-8') as file:
    raw_text = file.read()

print(f'Número total de caracteres: {len(raw_text)}.')
print(raw_text[:99])

Número total de caracteres: 20479.
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 



# Hace esto, línea por línea:

1.

```python
with open('./sample_data/llm_from_scratch/the-verdict.txt', 'r', encoding='utf-8') as file:
```

* `open(path, mode, encoding=...)`: abre un archivo y devuelve un objeto fichero.

  * `path`: `'./sample_data/llm_from_scratch/the-verdict.txt'`. Ruta relativa al **directorio de trabajo actual**. En Colab suele ser `/content`. `./` apunta ahí.
  * `mode='r'`: modo lectura de texto. No crea ni modifica el archivo.
  * `encoding='utf-8'`: decodifica bytes a texto Unicode con UTF-8. Evita errores de acentos.
* `with ... as file:` usa un **context manager**. Garantiza cierre del archivo al salir del bloque, incluso si hay excepciones. Evita fugas de descriptores.

2.

```python
    raw_text = file.read()
```

* `file.read()` lee **todo** el contenido en memoria y retorna un `str` (Unicode). Complejidad O(n).
* Útil para archivos pequeños/medianos. Para archivos muy grandes, preferir lectura por trozos o iterar líneas.

3.

```python
print(f'Número total de caracteres: {len(raw_text)}.')
```

* `len(raw_text)`: número de **caracteres Unicode** del `str`. No son bytes.
* Un “carácter” aquí es un **code point**. Algunos grafemas visibles pueden componerse de varios code points; `len` contaría más de uno en ese caso.
* `f'...'`: f-string. Evalúa expresiones dentro de `{}` y las inserta en el texto.

4.

```python
print(raw_text[:99])
```

* `raw_text[:99]`: **slice** desde el inicio hasta el índice 99 **excluido**. Devuelve como mucho 99 caracteres. Si el texto es más corto, no falla.
* Útil para previsualizar el inicio del archivo.

Notas prácticas en Colab:

* Verifica que el archivo exista en esa ruta. Si falla, `FileNotFoundError`.
* Si ves errores de codificación, confirma UTF-8 del archivo. Alternativa: `encoding='utf-8-sig'` si trae BOM, o `errors='replace'` para caracteres inválidos.
* Ruta alternativa robusta:

  ```python
  from pathlib import Path
  p = Path('sample_data/llm_from_scratch/the-verdict.txt')
  with p.open('r', encoding='utf-8') as f:
      raw_text = f.read()
  ```
* Si necesitas memoria estable, no uses `read()` con archivos muy grandes. Mejor:

  ```python
  size = 0
  with open(path, 'r', encoding='utf-8') as f:
      for line in f:
          size += len(line)
  ```

In [2]:
import re

text = 'Hey, you. This, is a test.'
result = re.split(r'\s', text)
print(result)

['Hey,', 'you.', 'This,', 'is', 'a', 'test.']


Explicación precisa línea por línea:

1.

```python
import re
```

* Importa el módulo estándar **`re`** (regular expressions). Permite búsqueda, coincidencia y manipulación de texto mediante **expresiones regulares**.
* Internamente compila patrones a autómatas finitos optimizados.

2.

```python
text = 'Hey, you. This, is a test.'
```

* Declara una variable `text` tipo `str` con el contenido literal indicado.
* Contiene palabras, comas, puntos y espacios.

3.

```python
result = re.split(r'\s', text)
```

* `re.split(pattern, string)` divide `string` en una lista, usando **las coincidencias del patrón** como separadores.
* `r'\s'` es un *raw string literal*: el prefijo `r` indica que `\` se trata como carácter literal y no como escape de Python.
* En una expresión regular, `\s` coincide con **cualquier carácter de espacio en blanco**: espacio, tabulador, salto de línea, retorno de carro o tab vertical.
* Por tanto, esta instrucción separa el texto cada vez que encuentra un espacio o cualquier carácter blanco.
* Devuelve una lista de subcadenas sin los delimitadores (los espacios no se incluyen en la salida).

4.

```python
print(result)
```

* Imprime la lista resultante.
* Salida esperada:

  ```
  ['Hey,', 'you.', 'This,', 'is', 'a', 'test.']
  ```

  Cada palabra o palabra con puntuación permanece unida porque `re.split` solo corta por espacios.

Detalles adicionales:

* Si el texto tuviera varios espacios seguidos, el resultado incluiría elementos vacíos (`''`) donde se encuentran separadores consecutivos.
* Ejemplo: `'a  b'` → `['a', '', 'b']`.
* Para evitar vacíos: `re.split(r'\s+', text.strip())`, que divide por uno o más espacios y elimina espacios iniciales/finales.


In [3]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hey', ',', '', ' ', 'you', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


# Explicación detallada:

1.

```python
result = re.split(r'([,.]|\s)', text)
```

* `re.split(patrón, texto)` divide `texto` usando las **coincidencias del patrón** como separadores.

* El patrón `r'([,.]|\s)'` se interpreta así:

  * `r'...'`: *raw string literal* evita que `\s` sea procesado como escape por Python.
  * `(...)`: **grupo de captura**. Si el patrón contiene paréntesis, los separadores encontrados se **conservan** en la lista de salida.
  * `[,.]`: clase de caracteres; coincide con **una coma (`,`) o un punto (`.`)**.
  * `|`: operador **OR** lógico en expresiones regulares.
  * `\s`: coincide con **cualquier espacio en blanco** (espacio, tabulador, salto de línea, etc.).

* En conjunto, el patrón significa:
  “Divide el texto cada vez que aparezca una coma, un punto o un espacio, e incluye ese carácter separador como elemento independiente en el resultado.”

* El grupo de captura `(...)` es lo que diferencia este ejemplo del anterior: sin él, los separadores se descartarían; con él, se **mantienen** en la lista.

2.

```python
print(result)
```

* Muestra la lista completa generada.
* Para `text = 'Hey, you. This, is a test.'`, el resultado es:

  ```
  ['Hey', ',', '', ' ', 'you', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
  ```

Análisis del resultado:

* Cada palabra o signo está separado individualmente.
* Los elementos vacíos (`''`) aparecen porque `re.split` puede generar subcadenas vacías cuando dos separadores se suceden (por ejemplo, una coma seguida de un espacio).
* Los separadores detectados (`,`, `.`, `' '`) se mantienen en la lista.

Uso posterior:

* Estos resultados suelen limpiarse con una lista por comprensión:

  ```python
  result = [item for item in result if item.strip()]
  ```

  que elimina los vacíos y deja solo palabras y signos de puntuación separados.


In [4]:
result = [item for item in result if item.strip()] # list comprehension
print(result)

['Hey', ',', 'you', '.', 'This', ',', 'is', 'a', 'test', '.']


Explicación exacta:

1.

```python
result = [item for item in result if item.strip()]
```

* Es una **lista por comprensión** (*list comprehension*).
* Recorre cada elemento `item` dentro de la lista `result` anterior.
* `item.strip()` ejecuta el método `str.strip()` de Python, que:

  * Elimina todos los caracteres de espacio en blanco al inicio y final del string (`' '`, `\t`, `\n`, etc.).
  * Devuelve el texto sin esos espacios.
* En un contexto booleano, un string vacío `''` evalúa como `False`.
* Así, la condición `if item.strip()` **filtra** y mantiene solo los elementos cuyo resultado no está vacío.
* Resultado: se eliminan todos los tokens vacíos (`''`) generados por separadores consecutivos o espacios redundantes.

2.

```python
print(result)
```

* Imprime la nueva lista limpia, sin elementos vacíos.
* Con `text = 'Hey, you. This, is a test.'`, la salida será:

  ```
  ['Hey', ',', 'you', '.', 'This', ',', 'is', 'a', 'test', '.']
  ```

Resultado final:

* Cada palabra y signo de puntuación es un token independiente.
* Se conserva el orden original del texto.
* Ya no hay espacios ni cadenas vacías.

Importancia:

* Este filtrado prepara el texto para construir vocabularios o convertir tokens en IDs numéricos, pasos previos al entrenamiento de un modelo de lenguaje.


In [5]:
text = 'Hey, you. Is ths-- a test?'
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)

result = [item for item in result if item.strip()]
print(result)

['Hey', ',', 'you', '.', 'Is', 'ths', '--', 'a', 'test', '?']


Explicación exhaustiva:

1.

```python
text = 'Hey, you. Is ths-- a test?'
```

* Define la variable `text` como una cadena (`str`).
* Contiene palabras, signos de puntuación, un doble guion (`--`) y espacios.
* Este tipo de texto se usa para probar una tokenización más compleja.

---

2.

```python
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
```

* Usa el método `re.split()` del módulo `re` para **dividir el texto** con una expresión regular avanzada.

* Desglose del patrón `r'([,.:;?_!"()\']|--|\s)'`:

  * `r'...'`: *raw string literal*, evita que Python interprete `\` como secuencia de escape.
  * `(...)`: **grupo de captura**, conserva los separadores en la salida.
  * `[ ,.:;?_!"()\' ]`: **clase de caracteres**, coincide con cualquiera de estos signos de puntuación:
    `, . : ; ? _ ! " ( ) '`
  * `|--`: el operador `|` indica “o”. Aquí captura explícitamente la secuencia de **doble guion** `--`.
  * `|\s`: también divide donde haya **espacios en blanco** (espacios, tabuladores o saltos de línea).

* En conjunto, el patrón significa:
  “Divide el texto en cada espacio o signo de puntuación del listado, y conserva esos signos como elementos separados en la lista resultante.”

---

3.

```python
result = [item for item in result if item.strip()]
```

* Lista por comprensión para eliminar tokens vacíos (`''`) o compuestos solo por espacios.
* `item.strip()` elimina espacios al inicio y al final; si el resultado está vacío, se descarta.
* Resultado: una lista de tokens limpios (palabras y signos).

---

4.

```python
print(result)
```

* Imprime la lista final de tokens.
* Salida esperada:

  ```
  ['Hey', ',', 'you', '.', 'Is', 'ths', '--', 'a', 'test', '?']
  ```

---

**Resumen funcional:**

* La expresión regular separa palabras y signos de puntuación en elementos individuales.
* El filtrado posterior elimina espacios vacíos.
* El resultado es una **tokenización básica** útil para construir vocabularios o convertir texto en secuencias de IDs numéricos en el preprocesamiento de un LLM.


# Tokenizando el texto



In [6]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


# Explicación exhaustiva, línea por línea:


### 1)

```python
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
```

**Propósito:**
Tokenizar el texto completo cargado en `raw_text` (por ejemplo, el cuento *The Verdict*).
Divide el texto en unidades mínimas (*tokens*) —palabras y signos de puntuación— que luego se usarán como entrada para un modelo de lenguaje.

**Desglose técnico:**

* **`re.split(patrón, cadena)`**
  Función del módulo estándar `re` (*regular expressions*) que divide una cadena (`cadena`) en una lista, usando las coincidencias del `patrón` como delimitadores.
  A diferencia de `str.split()`, que solo separa por espacios, `re.split()` puede usar expresiones regulares complejas y permite conservar los delimitadores si están dentro de **paréntesis de captura**.

* **`r'([,.:;?_!"()\']|--|\s)'`**
  Es un *raw string literal* (`r'...'`), lo que evita que Python interprete `\` como secuencia de escape (por ejemplo, `\s` no se convierte en “espacio”, sino que se pasa literalmente a la expresión regular).

  Desglose del patrón:

  * **`(...)`** → Grupo de captura. Todo lo que coincida dentro de este grupo se incluye en la lista de salida.
  * **`[,.:;?_!"()\' ]`** → Clase de caracteres que captura **cualquiera** de los signos de puntuación listados:
    `, . : ; ? _ ! " ( ) '`
  * **`|--`** → Alternativa literal que detecta la secuencia exacta de **doble guion** `--` (frecuente en narrativa).
  * **`|\s`** → Detecta cualquier carácter de espacio en blanco (`' '`, `\t`, `\n`, etc.).

  En conjunto, el patrón significa:

  > Divide el texto cada vez que aparezca una coma, punto, signo de interrogación, doble guion o espacio, e incluye esos separadores en la lista resultante.

**Resultado intermedio:**
La variable `preprocessed` se convierte en una lista en la que cada elemento puede ser:

* una palabra,
* un signo de puntuación aislado (`,`, `.`, `--`, etc.), o
* una cadena vacía si se encuentran separadores consecutivos.

---

### 2)

```python
preprocessed = [item.strip() for item in preprocessed if item.strip()]
```

**Propósito:**
Limpiar la lista de tokens eliminando elementos vacíos o compuestos solo por espacios.

**Explicación técnica:**

* `item.strip()` ejecuta el método `str.strip()` sobre cada `item`:

  * Quita caracteres de espacio en blanco al inicio y final del string.
  * Devuelve una nueva cadena sin esos caracteres.
* En un contexto booleano, un string vacío (`''`) se evalúa como `False`.
* Por tanto, la condición `if item.strip()` **filtra** y conserva solo los elementos con contenido textual o de puntuación válido.
* El resultado final es una lista limpia con todos los tokens “reales” que el modelo procesará.

**Complejidad temporal:**
O(n) respecto al número de tokens, ya que el filtrado recorre cada elemento una vez.

---

### 3)

```python
print(len(preprocessed))
```

**Propósito:**
Mostrar cuántos tokens (palabras y signos) contiene el texto tras la tokenización y limpieza.

**Detalles técnicos:**

* `len(preprocessed)` devuelve el número de elementos de la lista (un entero).
* Este número equivale a la **longitud del corpus tokenizado**.
* Permite comprobar si la división del texto produjo una cantidad razonable de tokens.

  * Por ejemplo, el cuento *The Verdict* produce ~4 690 tokens sin espacios.

**Consideraciones de uso:**

* Este conteo no equivale a palabras “semánticas”, ya que incluye signos de puntuación como tokens separados.
* El resultado sirve para determinar el tamaño del vocabulario base que se usará en la siguiente etapa (construcción del diccionario de tokens únicos).

---

**Resumen funcional global:**

1. Divide el texto en palabras y signos mediante una expresión regular amplia.
2. Elimina vacíos y espacios innecesarios.
3. Devuelve una lista limpia de tokens que representa la forma mínima procesable del corpus textual.
4. Muestra el número total de tokens para verificar la segmentación antes de construir el vocabulario del modelo.


# 2.3 Convirtiendo tokens en ID's

In [7]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

vocab = {token: integer for integer, token in enumerate(all_words)} # dict comprehension

for i, item in enumerate(vocab.items()):
  print(item)
  if i > 50:
    break


1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


Explicación exhaustiva, línea por línea:

---

### 1)

```python
all_words = sorted(set(preprocessed))
```

**Objetivo:**
Extraer el **vocabulario único** del corpus tokenizado `preprocessed` y ordenarlo alfabéticamente.

**Desglose técnico:**

* `set(preprocessed)`

  * Convierte la lista `preprocessed` (que contiene tokens repetidos) en un **conjunto** (`set`), una estructura de datos desordenada y sin duplicados.
  * Cada elemento del conjunto es **único**, por lo que esta operación elimina repeticiones.
  * Implementación interna: los `set` de Python están basados en **tablas hash**.

    * La inserción y la verificación de unicidad (`hashing`) tienen complejidad media **O(1)**.
  * Complejidad total de conversión: **O(n)**, donde *n* es el número de tokens en `preprocessed`.

* `sorted(...)`

  * Toma el conjunto resultante y devuelve una **lista ordenada lexicográficamente**.
  * La ordenación se realiza con el algoritmo **Timsort** (fusión adaptativa, estable y de complejidad O(n log n)).
  * El orden lexicográfico depende de los códigos Unicode de los caracteres, por lo que letras mayúsculas (`'A'`) aparecen antes que minúsculas (`'a'`).

**Resultado:**
Una lista (`list`) llamada `all_words` que contiene todos los **tokens únicos** (palabras y signos de puntuación), ordenados.

---

### 2)

```python
vocab_size = len(all_words)
```

**Objetivo:**
Calcular el tamaño del vocabulario, es decir, cuántos tokens distintos existen.

**Detalles:**

* `len()` accede al atributo de longitud de la lista (`PyObject_VAR_HEAD->ob_size` en CPython).
* Complejidad temporal: **O(1)**, ya que `len()` no recorre la lista; accede directamente al valor almacenado internamente.
* `vocab_size` es un entero (`int`) que representa el número total de tokens únicos.

---

### 3)

```python
print(vocab_size)
```

**Propósito:**
Mostrar la cantidad de elementos únicos (palabras, signos y símbolos) en el vocabulario.

* En el ejemplo del libro, para *The Verdict*, este valor es aproximadamente **1 130** tokens.
* Este número será fundamental para:

  * definir el tamaño de las matrices de embeddings,
  * establecer la dimensionalidad de las capas de entrada y salida del modelo (por ejemplo, `nn.Embedding(vocab_size, embed_dim)` en PyTorch).

---

### 4)

```python
vocab = {token: integer for integer, token in enumerate(all_words)}
```

**Objetivo:**
Crear un **diccionario de mapeo** entre tokens y sus identificadores numéricos (*token IDs*).

**Desglose técnico:**

* **`enumerate(all_words)`**

  * Genera un iterador que produce tuplas `(índice, elemento)` para cada token en `all_words`.
  * Por defecto, la enumeración comienza en 0.
  * Ejemplo: `[('!', 0), ('"', 1), ("'", 2), ('(', 3), ...]`.
  * Complejidad: **O(n)**, donde *n* es el número de tokens únicos.

* **Comprensión de diccionario `{token: integer for integer, token in enumerate(all_words)}`**

  * Recorre cada par `(integer, token)` producido por `enumerate`.
  * Asigna el token (`token`) como **clave** y el número (`integer`) como **valor**.
  * Resultado: un diccionario que implementa la función ( f : \text{token} \mapsto \text{ID entero} ).

**Propiedades del diccionario (`dict`):**

* En Python 3.7+, los diccionarios **mantienen el orden de inserción**, por lo que el orden coincide con la lista ordenada `all_words`.
* Internamente usan **tablas hash** (resolución abierta), lo que permite:

  * Búsqueda promedio: **O(1)**.
  * Inserción promedio: **O(1)**.
* Tamaño de `vocab` = `vocab_size`.

**Importancia:**
Este mapeo es el núcleo del proceso de **tokenización numérica**, que permite convertir texto en tensores de enteros antes del entrenamiento del modelo.

---

### 5)

```python
for i, item in enumerate(vocab.items()):
  print(item)
  if i > 50:
    break
```

**Objetivo:**
Verificar visualmente las primeras asignaciones del diccionario `vocab`.

**Desglose técnico:**

* **`vocab.items()`**

  * Devuelve un objeto *view* que itera sobre las **tuplas (clave, valor)** del diccionario.
  * Ejemplo: `('!', 0), ('"', 1), ("'", 2), ('(', 3), ...`.
  * Complejidad: **O(1)** por acceso, **O(n)** por recorrido completo.

* **`enumerate(...)`**

  * Agrega un contador `i` para poder detener el bucle después de imprimir 51 pares.

* **`print(item)`**

  * Muestra en consola cada par `(token, id)`.

* **Condición `if i > 50: break`**

  * Interrumpe el bucle tras 51 iteraciones (índices de 0 a 50).
  * Esto evita imprimir el vocabulario completo, que puede tener cientos o miles de tokens.

---

### **Resumen funcional completo:**

1. Se eliminan duplicados del corpus tokenizado.
2. Se ordenan los tokens para generar una lista estable y reproducible (`all_words`).
3. Se calcula el tamaño del vocabulario (`vocab_size`).
4. Se crea un diccionario `vocab` que asigna a cada token un identificador entero único, base de la codificación del texto.
5. Se imprime una muestra del mapeo para validación visual.

---

**Resultado conceptual:**
Este bloque construye la **primera capa de representación simbólica** de un LLM:

> un espacio discreto de vocabulario donde cada símbolo del lenguaje natural queda asociado a un índice entero, preparando el terreno para el embedding vectorial que convertirá estos índices en representaciones continuas en la siguiente etapa del pipeline.


In [8]:
class SimpleTokenizerV1:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i: s for s, i in vocab.items()} # ('his', 51) i= 51, s= his

  def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = ' '.join([self.int_to_str[i] for i in ids])

    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

    return text


# **Explicación exhaustiva del código y de los principios que implementa:**


## **Clase `SimpleTokenizerV1`**

Esta clase define un **tokenizador mínimo y bidireccional** que convierte texto en secuencias numéricas (*encoding*) y viceversa (*decoding*).
Opera sobre un vocabulario (`vocab`) previamente construido que asigna **tokens → IDs numéricos**.

La implementación ilustra el flujo esencial de un **preprocesador de texto para LLMs** antes de usar técnicas más avanzadas como *byte pair encoding (BPE)*.

---

### **1) Definición de clase**

```python
class SimpleTokenizerV1:
```

* Define un **objeto de tipo clase** en Python que agrupa datos (atributos) y comportamiento (métodos).
* Sirve como plantilla para crear instancias específicas del tokenizador.
* En tiempo de ejecución, cada instancia mantiene su propio estado interno (diccionarios de mapeo, etc.).

---

### **2) Método `__init__`**

```python
def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i: s for s, i in vocab.items()}  # ('his', 51) → i=51, s='his'
```

**Propósito:**
Inicializa la instancia y crea los dos diccionarios de mapeo complementarios:

1. **`str_to_int`**: de token (texto) a ID (entero).
2. **`int_to_str`**: de ID (entero) a token (texto).

**Detalles técnicos:**

* `__init__` es el **constructor** especial de Python. Se ejecuta automáticamente al crear un nuevo objeto:

  ```python
  tokenizer = SimpleTokenizerV1(vocab)
  ```

  Este llama implícitamente a `SimpleTokenizerV1.__init__(tokenizer, vocab)`.

* `self.str_to_int = vocab`

  * Almacena el diccionario que mapea cada token único del vocabulario a un entero (ID).
  * Ejemplo: `{'!', 0, '"': 1, "'", 2, ..., 'his': 51}`.

* `self.int_to_str = {i: s for s, i in vocab.items()}`

  * **Inversión del mapeo.**
  * `vocab.items()` produce pares `(token, id)`.
  * La comprensión de diccionario los invierte para formar `(id, token)`.
  * Ejemplo: `{51: 'his', 52: 'her', ...}`.
  * Esto permite reconstruir texto desde secuencias de IDs en la operación *decode*.
  * Complejidad de construcción: **O(n)** en el número de tokens.

---

### **3) Método `encode`**

```python
def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
```

**Función general:**
Convierte texto plano en una secuencia de **identificadores numéricos** según el vocabulario.

**Etapas detalladas:**

1. **Tokenización inicial**

   ```python
   preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
   ```

   * Usa una expresión regular para dividir el texto en **palabras y signos de puntuación**.
   * Patrón:

     * `(...)`: grupo de captura (mantiene los delimitadores).
     * `[,.?_!"()']`: signos a separar.
     * `|--`: doble guion literal.
     * `|\s`: espacio o tabulador.
   * Ejemplo:
     `'Hey, you!' → ['Hey', ',', '', ' ', 'you', '!', '']`

2. **Limpieza de tokens**

   ```python
   preprocessed = [item.strip() for item in preprocessed if item.strip()]
   ```

   * `str.strip()` elimina espacios en blanco al inicio y fin de cada token.
   * La condición `if item.strip()` elimina elementos vacíos (`''`).
   * Resultado limpio: `['Hey', ',', 'you', '!']`.

3. **Conversión a IDs**

   ```python
   ids = [self.str_to_int[s] for s in preprocessed]
   ```

   * Sustituye cada token textual por su ID numérico.
   * Acceso al diccionario: O(1) promedio por búsqueda (tabla hash).
   * Si un token no está en `vocab`, esta versión **lanza KeyError** (no maneja desconocidos).
   * Ejemplo:
     Si `vocab['Hey'] = 45`, `vocab[','] = 3`, la salida será `[45, 3, 120, 5]`.

4. **Retorno**

   * Devuelve una lista de enteros (`List[int]`).
   * Este formato es compatible con frameworks como PyTorch (`torch.tensor(ids)`).

---

### **4) Método `decode`**

```python
def decode(self, ids):
    text = ' '.join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text
```

**Función general:**
Transforma una secuencia numérica de tokens de vuelta a texto legible.

**Etapas detalladas:**

1. **Conversión de IDs a texto**

   ```python
   [self.int_to_str[i] for i in ids]
   ```

   * Usa el diccionario inverso `int_to_str` para reconstruir los tokens.
   * Genera una lista de strings.
   * Si un ID no está en el diccionario, se lanzará un `KeyError`.

2. **Concatenación con espacios**

   ```python
   ' '.join([...])
   ```

   * Une todos los tokens con un **espacio** entre ellos.
   * Ejemplo: `['Hey', ',', 'you', '!'] → 'Hey , you !'`.

3. **Corrección de espaciado antes de signos**

   ```python
   re.sub(r'\s+([,.?!"()\'])', r'\1', text)
   ```

   * Busca espacios seguidos de signos de puntuación.
   * Patrón:

     * `\s+`: uno o más espacios.
     * `([,.?!"()'])`: grupo de captura con signos.
   * Reemplazo `r'\1'`: mantiene solo el signo, eliminando el espacio anterior.
   * Resultado: `'Hey, you!'` (sin espacio extra antes de `,` o `!`).
   * Complejidad: O(n) sobre la longitud del texto.

4. **Retorno**

   * Devuelve el texto corregido y reconstruido como `str`.

---

### **5) Ejemplo de uso**

```python
tokenizer = SimpleTokenizerV1(vocab)
text = "It's the last he painted, you know."
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))
```

**Flujo interno:**

1. `encode()` → tokeniza → mapea a IDs.
2. `decode()` → convierte IDs → reconstruye texto.

**Resultado esperado:**

```python
[12, 45, 78, 91, 8, 120, 5, 32, 14]  # ejemplo de IDs
"It's the last he painted, you know."
```

---

### **6) Limitaciones del diseño V1**

* No maneja **palabras desconocidas**: si un token no está en el vocabulario, lanza `KeyError`.
* No incluye **tokens especiales** (`<|unk|>`, `<|endoftext|>`, etc.).
* No conserva espacios exactos, por lo que pierde formato o saltos de línea.
* Sirve como **prototipo conceptual** antes de evolucionar hacia `SimpleTokenizerV2` y finalmente hacia el **BPE tokenizer** usado en GPT.

---

### **Resumen conceptual**

| Etapa    | Entrada      | Proceso                             | Salida        | Complejidad |
| -------- | ------------ | ----------------------------------- | ------------- | ----------- |
| `encode` | texto crudo  | regex + limpieza + mapeo            | lista de IDs  | O(n)        |
| `decode` | lista de IDs | reconstrucción + ajuste de espacios | texto legible | O(n)        |

El objetivo pedagógico de `SimpleTokenizerV1` es mostrar **cómo un LLM traduce lenguaje natural a un espacio discreto numérico** antes de aplicar embeddings y atención. Es la base de la representación simbólica que alimenta las capas iniciales del modelo transformer.


In [9]:
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know, "Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


Explicación exhaustiva, línea por línea:

---

### **1)**

```python
tokenizer = SimpleTokenizerV1(vocab)
```

**Propósito:**
Crea una **instancia** de la clase `SimpleTokenizerV1` definida previamente, usando como entrada el diccionario `vocab`.

**Detalles técnicos:**

* `SimpleTokenizerV1(vocab)` ejecuta internamente:

  ```python
  SimpleTokenizerV1.__init__(tokenizer, vocab)
  ```

  donde `tokenizer` es el nuevo objeto creado en memoria (tipo `SimpleTokenizerV1`).

* En el constructor (`__init__`):

  * `self.str_to_int = vocab` asigna el vocabulario base (tokens → IDs).
  * `self.int_to_str = {i: s for s, i in vocab.items()}` crea el diccionario inverso (IDs → tokens).

**Estructura interna resultante:**

* `tokenizer.str_to_int` → `{'!': 0, '"': 1, ... 'painted': 418, ...}`
* `tokenizer.int_to_str` → `{0: '!', 1: '"', ... 418: 'painted', ...}`

Estas estructuras residen en la memoria heap y son accedidas por referencia a través del objeto `tokenizer`.

**Complejidad:**

* Construcción del diccionario inverso: **O(n)** donde *n* es el tamaño del vocabulario.

---

### **2)**

```python
text = """Its the last he painted, you know, "Mrs. Gisburn said with pardonable pride."""
```

**Propósito:**
Define el texto que será convertido en tokens y luego a IDs.

**Detalles:**

* Uso de triple comillas `"""..."""` permite escribir cadenas multilínea sin necesidad de escapes.
  Aquí se usa en una sola línea, pero sigue siendo válido.
* El texto incluye:

  * palabras (`Its`, `the`, `last`, `painted`, `know`, `said`, etc.),
  * signos de puntuación (`,`, `"`, `.`),
  * un nombre propio con mayúscula (`Mrs. Gisburn`).

**Nota importante:**
El token `"Its"` (sin apóstrofe) podría **no existir en el vocabulario** si el texto de entrenamiento original solo contenía `"It's"`.
Dado que `SimpleTokenizerV1` **no maneja palabras desconocidas**, esto puede causar un `KeyError`.

---

### **3)**

```python
ids = tokenizer.encode(text)
```

**Propósito:**
Convierte el texto en una secuencia de enteros (*token IDs*) siguiendo el vocabulario cargado.

---

#### **Etapas internas del método `encode`:**

1. **Tokenización inicial**

   ```python
   preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
   ```

   * Divide el texto en palabras, puntuaciones y espacios.
   * Cada separador se conserva como elemento separado.
   * Ejemplo intermedio:

     ```
     ['Its', ' ', 'the', ' ', 'last', ' ', 'he', ' ', 'painted', ',', ' ', 'you', ' ', 'know', ',', ' ', '"', 'Mrs', '.', ' ', 'Gisburn', ' ', 'said', ' ', 'with', ' ', 'pardonable', ' ', 'pride', '.', '', '']
     ```

2. **Limpieza**

   ```python
   preprocessed = [item.strip() for item in preprocessed if item.strip()]
   ```

   * Elimina cadenas vacías y espacios redundantes.
   * Resultado limpio:

     ```
     ['Its', 'the', 'last', 'he', 'painted', ',', 'you', 'know', ',', '"', 'Mrs', '.', 'Gisburn', 'said', 'with', 'pardonable', 'pride', '.']
     ```

3. **Conversión a IDs**

   ```python
   ids = [self.str_to_int[s] for s in preprocessed]
   ```

   * Busca cada token en el diccionario `str_to_int`.
   * Si todos los tokens están presentes en `vocab`, se obtiene una lista de enteros:

     ```
     [315, 27, 46, 89, 418, 3, 57, 108, 3, 1, 302, 7, 410, 85, 92, 600, 612, 7]
     ```

     *(Los valores son ilustrativos; dependen del vocabulario real.)*
   * Si un token no existe (p. ej., `"Its"` en lugar de `"It's"`), Python lanzará:

     ```
     KeyError: 'Its'
     ```

**Razonamiento del error:**

* El vocabulario proviene del cuento *The Verdict*, que probablemente no contiene `"Its"`.
* La versión `"It's"` (con apóstrofe) sí estaría registrada como un token distinto.
* Por tanto, el tokenizador V1 falla con palabras fuera del conjunto de entrenamiento.

**Complejidad:**

* Búsqueda en diccionario para *n* tokens → **O(n)** promedio, ya que cada acceso hash es O(1).

---

### **4)**

```python
print(ids)
```

**Propósito:**
Muestra la lista numérica resultante, útil para validar la conversión.

* Si todos los tokens existen: imprime los **índices numéricos** del texto.
  Ejemplo hipotético:

  ```
  [315, 27, 46, 89, 418, 3, 57, 108, 3, 1, 302, 7, 410, 85, 92, 600, 612, 7]
  ```
* Si hay palabras desconocidas: no imprime nada y muestra un **KeyError** interrumpiendo la ejecución.

---

### **Conclusión operativa**

| Paso                       | Acción                    | Resultado                     | Complejidad |
| -------------------------- | ------------------------- | ----------------------------- | ----------- |
| `SimpleTokenizerV1(vocab)` | Crea mapeos token↔ID      | Diccionarios en memoria       | O(n)        |
| `encode(text)`             | Divide, limpia, convierte | Lista de IDs                  | O(m)        |
| `print(ids)`               | Visualiza los IDs         | Validación de la codificación | O(m)        |

---

### **Limitación clave evidenciada**

El **modelo V1 no soporta palabras desconocidas**.
Por tanto, si `KeyError: 'Its'` ocurre, se debe usar `SimpleTokenizerV2`, que reemplaza tokens no vistos con `<|unk|>` (token de desconocido), asegurando compatibilidad con cualquier texto de entrada.


In [10]:
print(tokenizer.decode(ids))

It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


Explicación exhaustiva:

---

### **1)**

```python
print(tokenizer.decode(ids))
```

**Propósito:**
Reconstruir texto legible a partir de la secuencia de identificadores numéricos `ids` obtenidos con `encode()`, y mostrarlo en pantalla.

---

## **Desglose interno del método `decode()`**

El método definido en la clase `SimpleTokenizerV1`:

```python
def decode(self, ids):
    text = ' '.join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text
```

### **Etapa 1 — Reconversión de IDs a tokens**

```python
[self.int_to_str[i] for i in ids]
```

* Usa el diccionario inverso `int_to_str`, construido durante la inicialización:

  ```python
  self.int_to_str = {i: s for s, i in vocab.items()}
  ```
* Para cada entero `i` en la lista `ids`, obtiene el token textual original asociado.

**Ejemplo:**

```
ids = [315, 27, 46, 89, 418, 3, 57, 108, 3, 1, 302, 7, 410, 85, 92, 600, 612, 7]
↓
['Its', 'the', 'last', 'he', 'painted', ',', 'you', 'know', ',', '"', 'Mrs', '.', 'Gisburn', 'said', 'with', 'pardonable', 'pride', '.']
```

* Si algún ID no está en el diccionario (poco probable, ya que `encode()` solo usa IDs válidos), Python lanza `KeyError`.

* Complejidad temporal: **O(n)** en el número de IDs.

---

### **Etapa 2 — Unión de tokens con espacios**

```python
text = ' '.join([...])
```

* Concatena todos los tokens, separándolos con un **espacio**.
* El resultado inicial conserva un espacio entre cada token, incluso antes de los signos de puntuación.

**Ejemplo intermedio:**

```
'Its the last he painted , you know , " Mrs . Gisburn said with pardonable pride .'
```

* Esta forma contiene **espacios no naturales** antes de comas, puntos o comillas.

---

### **Etapa 3 — Corrección tipográfica de espacios**

```python
text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
```

**Análisis de la expresión regular:**

* `re.sub(patrón, reemplazo, texto)` busca todas las coincidencias de `patrón` y las sustituye por `reemplazo`.

* Patrón: `r'\s+([,.?!"()\'])'`

  * `\s+` → uno o más espacios.
  * `([,.?!"()'])` → grupo de captura que coincide con cualquier signo de puntuación listado:
    coma, punto, interrogación, exclamación, comillas, paréntesis o apóstrofe.

* Reemplazo: `r'\1'`

  * Sustituye toda la coincidencia (espacios + signo) por **solo el signo**, manteniendo lo que se capturó entre paréntesis.
  * Resultado: elimina los espacios que preceden a la puntuación.

**Ejemplo de transformación:**

```
'Its the last he painted , you know , " Mrs . Gisburn said with pardonable pride .'
↓
'Its the last he painted, you know, "Mrs. Gisburn said with pardonable pride.'
```

**Resultado:**

* Se eliminan espacios antes de los signos, pero se conservan los espacios correctos entre palabras.
* El texto recupera una forma natural y gramaticalmente correcta.

---

### **Etapa 4 — Retorno y visualización**

```python
return text
```

* Devuelve la cadena corregida al punto de llamada.
* `print(tokenizer.decode(ids))` muestra el resultado final.

**Salida esperada:**

```
Its the last he painted, you know, "Mrs. Gisburn said with pardonable pride."
```

---

## **Resumen de flujo de datos**

| Fase | Entrada                  | Operación                               | Salida                         | Complejidad |
| ---- | ------------------------ | --------------------------------------- | ------------------------------ | ----------- |
| 1    | `ids` (lista de enteros) | Mapea IDs → tokens                      | lista de strings               | O(n)        |
| 2    | lista de tokens          | Une con `' '.join()`                    | texto con espacios redundantes | O(n)        |
| 3    | texto con espacios       | `re.sub()` elimina espacios incorrectos | texto limpio                   | O(n)        |
| 4    | texto limpio             | `print()`                               | visualización                  | O(1)        |

---

## **Limitación funcional**

* Si algún token ID no tiene correspondencia en `int_to_str`, el método genera un error.
* `SimpleTokenizerV1` **no distingue** entre comillas de apertura y cierre, ni conserva saltos de línea o tabulaciones originales.
* No maneja tokens especiales como `<|endoftext|>` o `<|unk|>`.
* El resultado textual está diseñado para **coherencia lingüística básica**, no para reconstrucción exacta de formato.

---

**En síntesis:**
`decode()` implementa el paso inverso de `encode()` y ejemplifica el concepto de **reversibilidad parcial de tokenización**: a partir de una secuencia de índices numéricos, se puede recrear texto humano-legible mediante un mapeo inverso y una corrección superficial de espacios, etapa fundamental antes de evaluar la calidad de generación de un LLM.


In [19]:
# Aquí esperamos un error:

text = 'Hi, do you lik tea?' # Hi es una palabra no contenida en el vocabulario generado en lso pasos anteriores
print(tokenizer.encode(text))

KeyError: 'Hi'

Explicación exhaustiva:

---

### **1)**

```python
text = 'Hi, do you lik tea?'
```

**Propósito:**
Definir una nueva cadena de texto para comprobar el comportamiento del tokenizador cuando aparecen **palabras fuera del vocabulario**.

**Contexto técnico:**
El vocabulario `vocab` se generó a partir del cuento *The Verdict* de Edith Wharton.
Ese corpus no contiene todas las palabras del inglés; por ejemplo, es probable que no incluya:

* `"Hi"` (saludo coloquial),
* `"lik"` (error ortográfico de `"like"`).

Por tanto, estas palabras estarán **fuera del conjunto de tokens conocidos**.

**Implicación:**
El tokenizador actual (`SimpleTokenizerV1`) no maneja “out-of-vocabulary” (OOV) tokens.
Intentar convertir un token desconocido a su ID provocará una excepción de tipo `KeyError`.

---

### **2)**

```python
print(tokenizer.encode(text))
```

**Propósito:**
Llamar al método `encode()` para tokenizar y convertir el texto en IDs numéricos, e imprimir el resultado.

---

## **Flujo interno dentro de `encode()`**

```python
def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
```

---

### **Etapa 1 — División del texto**

```python
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
```

* Usa una expresión regular para dividir el texto por signos de puntuación y espacios, manteniendo los separadores.
* Resultado intermedio aproximado:

  ```
  ['Hi', ',', '', ' ', 'do', ' ', 'you', ' ', 'lik', ' ', 'tea', '?', '']
  ```

---

### **Etapa 2 — Limpieza**

```python
preprocessed = [item.strip() for item in preprocessed if item.strip()]
```

* Elimina espacios y cadenas vacías, dejando solo tokens válidos.
* Resultado limpio:

  ```
  ['Hi', ',', 'do', 'you', 'lik', 'tea', '?']
  ```

---

### **Etapa 3 — Conversión a IDs**

```python
ids = [self.str_to_int[s] for s in preprocessed]
```

* Intenta acceder a cada token dentro del diccionario `self.str_to_int` (el vocabulario).
* El tokenizador busca:

  * `vocab['Hi']`
  * `vocab[',']`
  * `vocab['do']`
  * `vocab['you']`
  * `vocab['lik']`
  * `vocab['tea']`
  * `vocab['?']`

**Resultado técnico:**

* Si **todos** los tokens están presentes, devuelve una lista de enteros (`List[int]`).
* En este caso, `"Hi"` y `"lik"` **no existen** en el vocabulario.
* En cuanto el intérprete intenta acceder a `self.str_to_int['Hi']`, el diccionario lanza:

  ```python
  KeyError: 'Hi'
  ```

---

### **Etapa 4 — Impresión**

* La ejecución se interrumpe antes de llegar a `print()`.
* Python muestra la traza del error:

  ```
  KeyError: 'Hi'
  ```

---

## **Análisis del error**

**Tipo de error:**
`KeyError` — ocurre cuando se intenta acceder a una clave inexistente en un diccionario.

**Razón interna:**

* En `SimpleTokenizerV1`, el método `encode()` no incluye ningún manejo de excepciones.
* El acceso directo `self.str_to_int[s]` depende de que todos los tokens estén presentes.
* Python, al no encontrar la clave `'Hi'` en la tabla hash interna del diccionario `self.str_to_int`, lanza la excepción.

**Comportamiento esperado (salida):**

```
KeyError: 'Hi'
```

---

## **Solución conceptual (adelanto del siguiente paso del libro)**

Para manejar palabras desconocidas, se usa una versión extendida del tokenizador:

```python
class SimpleTokenizerV2:
    def encode(self, text):
        ...
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
```

* Introduce un **token especial `<|unk|>`** (*unknown token*).
* Cada palabra fuera del vocabulario se reemplaza automáticamente por ese símbolo.
* Así, el método nunca falla y puede codificar cualquier texto.

**Ejemplo con `SimpleTokenizerV2`:**

```
text = 'Hi, do you lik tea?'
↓
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?']
↓
[1160, 3, 242, 57, 1160, 642, 10]
```

*(donde `1160` es el ID asignado a `<|unk|>` en el vocabulario extendido)*

---

## **Resumen**

| Etapa        | Acción                  | Resultado            | Complejidad | Observación                  |   |                    |
| ------------ | ----------------------- | -------------------- | ----------- | ---------------------------- | - | ------------------ |
| Tokenización | `re.split`              | Palabras y signos    | O(n)        | Separa espacios y puntuación |   |                    |
| Limpieza     | `strip` + filtro        | Tokens válidos       | O(n)        | Elimina vacíos               |   |                    |
| Mapeo a IDs  | Búsqueda en diccionario | `KeyError` en `'Hi'` | O(n)        | Sin manejo OOV               |   |                    |
| Solución     | Usar `<                 | unk                  | >` token    | Codificación robusta         | — | Implementado en V2 |

---

**Conclusión:**
El comando `print(tokenizer.encode(text))` con `SimpleTokenizerV1` falla porque el tokenizador no reconoce “Hi” ni “lik”.
Este comportamiento es **intencional** en la versión V1 del libro: su propósito es demostrar la necesidad de **ampliar el vocabulario** o **introducir un mecanismo de manejo de palabras desconocidas**, lo que se resolverá en la siguiente versión (`SimpleTokenizerV2`).


# **2.4 Adding special context tokens** (Agregando tokens de contexto especiales)

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(['<|endoftext|>', '<|unk|>'])
vocab = {token: integer for integer, token in enumerate(all_tokens)}

print(len(vocab.items()))

Explicación exhaustiva, línea por línea:

---

### **1)**

```python
all_tokens = sorted(list(set(preprocessed)))
```

**Propósito:**
Construir la base del **nuevo vocabulario** de tokens, eliminando duplicados y ordenándolos alfabéticamente.

**Desglose técnico:**

* `set(preprocessed)`

  * Convierte la lista `preprocessed` en un **conjunto** (`set`), eliminando repeticiones.
  * Los conjuntos en Python están implementados como **tablas hash**, lo que permite verificar y almacenar unicidad en tiempo promedio **O(1)** por operación.
  * Resultado: todos los tokens únicos del corpus.

* `list(...)`

  * Convierte el conjunto de nuevo en una lista indexable.
  * Esto es necesario porque los conjuntos son desordenados y no soportan orden ni indexación.

* `sorted(...)`

  * Ordena lexicográficamente la lista resultante según el código Unicode de cada carácter.
  * El algoritmo usado es **Timsort** (estable, O(n log n)).
  * El resultado es reproducible (misma ordenación cada vez), lo cual es crucial para que los IDs sean consistentes en diferentes ejecuciones.

**Resultado:**
Una lista `all_tokens` con todos los tokens únicos del texto base (`preprocessed`), ordenados de manera determinista.

---

### **2)**

```python
all_tokens.extend(['<|endoftext|>', '<|unk|>'])
```

**Propósito:**
Agregar dos **tokens especiales** al vocabulario.

**Desglose conceptual:**

* `.extend(lista)`

  * Añade varios elementos al final de la lista existente (modificación in-place).
  * Complejidad O(k), donde *k* es el número de elementos añadidos (aquí, 2).

**Tokens agregados:**

1. **`<|endoftext|>`**

   * Marca el **final de una secuencia textual**.
   * Se usa durante el entrenamiento y la generación de texto para indicar cuándo detener la predicción.
   * Análogo al token `[EOS]` (*End of Sequence*) en otros modelos.

2. **`<|unk|>`**

   * Representa cualquier **token desconocido** (*unknown token*).
   * Es el símbolo que sustituirá palabras no vistas durante la fase de entrenamiento (manejo OOV, *Out-Of-Vocabulary*).
   * En la práctica, este token permite que el modelo funcione sobre cualquier texto, incluso si aparecen palabras nuevas.

**Resultado:**
`all_tokens` contiene ahora:

```
[... 'yourself', '<|endoftext|>', '<|unk|>']
```

Si el vocabulario original tenía *N* tokens, ahora tendrá *N + 2*.

---

### **3)**

```python
vocab = {token: integer for integer, token in enumerate(all_tokens)}
```

**Propósito:**
Generar un **diccionario de mapeo** entre cada token y un identificador numérico único (índice entero).

**Desglose técnico:**

* `enumerate(all_tokens)`

  * Devuelve un iterador que produce pares `(índice, token)`, donde el índice empieza en 0.
  * Ejemplo:

    ```
    0: '!', 1: '"', 2: "'", ..., 1127: '<|endoftext|>', 1128: '<|unk|>'
    ```

* La comprensión de diccionario:

  ```python
  {token: integer for integer, token in enumerate(all_tokens)}
  ```

  * Invierte el orden del par, de `(índice, token)` a `(token, índice)`.
  * Cada token textual se convierte en **clave**, y su posición en **valor entero**.
  * Implementación interna: el diccionario (`dict`) es una tabla hash con búsqueda promedio O(1).

**Importancia:**

* Este diccionario constituye el **vocabulario formal del modelo**.
* El token `<|unk|>` permitirá asignar un valor por defecto a cualquier palabra fuera del vocabulario conocido.
* El token `<|endoftext|>` servirá como marcador de final de secuencia en el entrenamiento autoregresivo del LLM.

---

### **4)**

```python
print(len(vocab.items()))
```

**Propósito:**
Verificar el tamaño total del vocabulario extendido.

**Detalles técnicos:**

* `vocab.items()` devuelve una *vista* (`dict_items`) con todos los pares `(token, id)` del diccionario.
* `len(vocab.items())` cuenta cuántos pares existen.
* Este valor equivale al **número total de tokens únicos + 2** (por los tokens especiales añadidos).

**Ejemplo de salida:**

```
1130
```

si el vocabulario original tenía 1128 tokens únicos.

**Complejidad temporal:**

* `len()` sobre una estructura `dict` o `dict_items` es O(1), ya que el tamaño se almacena internamente en el encabezado del objeto.

---

### **Resumen funcional**

| Línea                       | Acción                   | Resultado              | Complejidad |
| --------------------------- | ------------------------ | ---------------------- | ----------- |
| `set(preprocessed)`         | Eliminar duplicados      | Tokens únicos          | O(n)        |
| `sorted(list(...))`         | Ordenar tokens           | Lista ordenada         | O(n log n)  |
| `.extend([...])`            | Añadir tokens especiales | Lista +2 elementos     | O(1)        |
| Diccionario por comprensión | Crear mapeo token→ID     | Diccionario `vocab`    | O(n)        |
| `len(vocab.items())`        | Contar tokens totales    | Tamaño del vocabulario | O(1)        |

---

### **Concepto general**

Este bloque crea un **vocabulario robusto**, apto para usarse en la versión mejorada del tokenizador (`SimpleTokenizerV2`):

* Garantiza consistencia (orden determinista).
* Soporta tokens fuera del vocabulario (`<|unk|>`).
* Integra delimitadores de secuencia (`<|endoftext|>`).
* Es la base para entrenar embeddings en modelos LLM de tipo GPT o Transformer decoder.

En síntesis:

> Aquí se define el *espacio discreto completo de símbolos* que el modelo será capaz de representar, asegurando que todo texto —conocido o no— pueda codificarse de forma numérica sin errores.


In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
  print(item)

Explicación exhaustiva, línea por línea:

---

### **1)**

```python
for i, item in enumerate(list(vocab.items())[-5:]):
```

**Propósito:**
Recorrer e imprimir los **últimos cinco elementos** del diccionario `vocab`, mostrando los tokens y sus identificadores numéricos más altos (generalmente los tokens añadidos al final, como `<|endoftext|>` y `<|unk|>`).

---

#### **Desglose técnico:**

##### a) `vocab.items()`

* Devuelve una **vista de elementos** del diccionario (`dict_items`), que contiene todos los pares `(clave, valor)` del diccionario `vocab`.

  * Cada elemento es una tupla: `(token, id_entero)`.
  * Ejemplo:

    ```
    ('yourself', 1127), ('<|endoftext|>', 1128), ('<|unk|>', 1129)
    ```

* En Python 3.7+, los diccionarios **preservan el orden de inserción**, por lo que los últimos elementos son los últimos tokens añadidos a `all_tokens`.

##### b) `list(vocab.items())`

* Convierte la vista `dict_items` en una **lista indexable**.
* Esto permite aplicar slicing (`[-5:]`) para acceder a los últimos cinco elementos.
* Complejidad temporal: **O(n)**, ya que se copian todas las entradas del diccionario a una nueva lista.

##### c) `[-5:]`

* Es un **slice** (rebanado) que selecciona los últimos cinco elementos de la lista.
* Ejemplo:

  ```python
  list(vocab.items())[-5:]
  # → [('your', 1125), ('yourself', 1126), ('<|endoftext|>', 1127), ('<|unk|>', 1128)]
  ```
* Si el diccionario tiene menos de 5 elementos, devuelve todos.

##### d) `enumerate(...)`

* Añade un contador `i` que empieza en 0 por defecto.

* Devuelve pares `(i, item)`, donde:

  * `i` es el índice del elemento en el bucle,
  * `item` es la tupla `(token, id)`.

* Ejemplo:

  ```
  0 ('your', 1125)
  1 ('yourself', 1126)
  2 ('<|endoftext|>', 1127)
  3 ('<|unk|>', 1128)
  ```

**Complejidad general del bucle:**

* O(n) por la conversión `list(vocab.items())`, pero solo recorre 5 elementos en la iteración.

---

### **2)**

```python
print(item)
```

**Propósito:**
Imprimir cada tupla `(token, id)` generada por el bucle.

* Cada línea de salida mostrará un par token–índice del vocabulario.
* Ejemplo de salida real:

  ```
  ('your', 1125)
  ('yourself', 1126)
  ('<|endoftext|>', 1127)
  ('<|unk|>', 1128)
  ```

**Detalles técnicos de `print`:**

* `print()` convierte cada objeto en su representación de texto (`str()` o `repr()`) y lo envía al flujo estándar de salida (`sys.stdout`).
* Cada llamada termina en salto de línea `\n` por defecto.

---

### **Resumen funcional**

| Etapa            | Acción                    | Resultado              | Complejidad |
| ---------------- | ------------------------- | ---------------------- | ----------- |
| `vocab.items()`  | Obtener pares (token, id) | vista `dict_items`     | O(1)        |
| `list(...)`      | Convertir a lista         | lista indexable        | O(n)        |
| `[-5:]`          | Tomar últimos 5           | sublista               | O(1)        |
| `enumerate(...)` | Añadir índice             | pares (i, (token, id)) | O(1)        |
| `print(item)`    | Mostrar resultado         | salida en consola      | O(1)        |

---

### **Resultado conceptual**

Este fragmento sirve como verificación visual del **final del vocabulario** y confirma que los **tokens especiales** (`<|endoftext|>`, `<|unk|>`) fueron correctamente añadidos y asignados con los índices más altos.

En contexto, los últimos valores impresos deben ser similares a:

```
('your', 1125)
('yourself', 1126)
('<|endoftext|>', 1127)
('<|unk|>', 1128)
```

Esto confirma que:

* El vocabulario está ordenado lexicográficamente.
* Los dos tokens especiales fueron añadidos al final de la lista.
* Los índices son consecutivos y únicos.

En síntesis: este bucle actúa como **validación final de la integridad del vocabulario** antes de proceder a la implementación del `SimpleTokenizerV2`.


# **Listing 2.4 A simple text tokenizer that handles unknown words**
# **Listado 2.4 Un tokenizador de texto simple que maneja palabras desconocidas**

In [ ]:
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i: s for s, i in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])

    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text


Explicación exhaustiva del diseño y comportamiento de `SimpleTokenizerV2`, línea por línea y a nivel conceptual:

---

## **Definición general**

```python
class SimpleTokenizerV2:
```

**Propósito:**
Implementar una versión **tolerante a palabras desconocidas** (*Out-Of-Vocabulary*, OOV) del tokenizador simple desarrollado previamente (`SimpleTokenizerV1`).

**Contexto técnico:**

* La versión anterior lanzaba un `KeyError` cuando un token no existía en el vocabulario.
* Esta nueva versión introduce un **mecanismo de respaldo** (`<|unk|>`) que permite procesar cualquier entrada textual.
* Representa la transición entre un tokenizador experimental y uno funcionalmente robusto para entrenamiento real de modelos.

---

## **1) Constructor**

```python
def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i: s for s, i in vocab.items()}
```

### **Propósito:**

Inicializar los mapeos **token → id** y **id → token** para permitir conversión bidireccional.

### **Detalles técnicos:**

* `self.str_to_int = vocab`

  * Guarda una referencia al diccionario base que mapea cadenas (tokens) a índices enteros.
  * Este diccionario contiene ahora también los tokens especiales `<|endoftext|>` y `<|unk|>`.
  * Acceso promedio: **O(1)** (tabla hash).

* `self.int_to_str = {i: s for s, i in vocab.items()}`

  * Inversión de claves y valores para decodificación.
  * Construye un nuevo diccionario: cada id entero apunta a su token textual.
  * Ejemplo:

    ```
    {0: '!', 1: '"', ..., 1127: '<|endoftext|>', 1128: '<|unk|>'}
    ```
  * Complejidad de construcción: **O(n)** donde *n* es el número de tokens.
  * Requiere recorrer el vocabulario completo una vez.

**Resultado:**
Dos estructuras complementarias que permiten conversión directa entre representación textual y numérica.

---

## **2) Método `encode()`**

```python
def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
```

### **Propósito:**

Convertir texto crudo (`str`) en una secuencia de identificadores numéricos (`List[int]`), asignando `<|unk|>` a los tokens no presentes en el vocabulario.

---

### **Etapa 1 — Tokenización base**

```python
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
```

* Usa la misma expresión regular que en la versión V1 para segmentar texto:

  * `(...)` grupo de captura → conserva los separadores.
  * `[,.?_!"()']` → coincide con signos de puntuación básicos.
  * `|--` → detecta doble guion literal.
  * `|\s` → divide por cualquier espacio en blanco.
* Genera una lista con palabras, signos y separadores vacíos.
* Complejidad: **O(n)** sobre el número de caracteres.

---

### **Etapa 2 — Limpieza de tokens**

```python
preprocessed = [item.strip() for item in preprocessed if item.strip()]
```

* `item.strip()` elimina espacios al inicio y al final del token.

* El `if` descarta cadenas vacías resultantes.

* Resultado: una lista limpia con palabras y signos.

* Ejemplo:

  ```
  ['Hi', ',', 'do', 'you', 'lik', 'tea', '?']
  ```

* Complejidad: **O(m)**, donde *m* = número de tokens.

---

### **Etapa 3 — Manejo de palabras desconocidas**

```python
preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
```

* Verifica para cada token si existe en el vocabulario (`self.str_to_int`):

  * Si **existe**, lo mantiene igual.
  * Si **no existe**, lo reemplaza por el token especial `<|unk|>`.
* Esto evita excepciones `KeyError` y asegura que todos los tokens puedan convertirse en IDs.
* Implementación:

  * Búsqueda hash promedio O(1) por token.
  * Complejidad total de esta etapa: O(m).

**Ejemplo:**

```python
['Hi', ',', 'do', 'you', 'lik', 'tea', '?']
↓
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?']
```

---

### **Etapa 4 — Conversión a IDs**

```python
ids = [self.str_to_int[s] for s in preprocessed]
```

* Sustituye cada token textual por su identificador numérico.

* Como todos los tokens ya están garantizados en el vocabulario, no hay errores.

* Ejemplo hipotético:

  ```
  {'<|unk|>': 1128, ',': 3, 'do': 42, 'you': 57, 'tea': 89, '?': 9}
  ↓
  [1128, 3, 42, 57, 1128, 89, 9]
  ```

* Complejidad temporal: **O(m)**.

* Complejidad espacial: O(m) para la lista de salida.

---

### **Etapa 5 — Retorno**

```python
return ids
```

* Devuelve una lista de enteros (`List[int]`).
* Esta lista es adecuada para ser convertida en tensor de entrada en frameworks de entrenamiento (PyTorch, TensorFlow).

---

## **3) Método `decode()`**

```python
def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text
```

### **Propósito:**

Convertir una lista de identificadores numéricos en texto legible, aplicando corrección de espacios.

---

### **Etapa 1 — Reconstrucción de tokens**

```python
[self.int_to_str[i] for i in ids]
```

* Usa el diccionario inverso `int_to_str` para traducir cada ID a su token textual.
* Complejidad: **O(m)**, acceso hash promedio O(1) por elemento.
* Resultado intermedio (lista de strings).
  Ejemplo:

  ```
  ['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?']
  ```

---

### **Etapa 2 — Unión con espacios**

```python
text = " ".join([...])
```

* Concatena los tokens con un espacio como separador.

* Ejemplo:

  ```
  '<|unk|> , do you <|unk|> tea ?'
  ```

* Complejidad: O(m) respecto a longitud del texto.

---

### **Etapa 3 — Limpieza tipográfica**

```python
text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
```

* Usa una expresión regular para eliminar espacios redundantes antes de signos de puntuación.
* Patrón:

  * `\s+` → uno o más espacios.
  * `([,.?!"()'])` → grupo de signos de puntuación comunes.
* Reemplazo: `r'\1'` → mantiene solo el signo, eliminando el espacio previo.
* Ejemplo:

  ```
  '<|unk|> , do you <|unk|> tea ?'
  ↓
  '<|unk|>, do you <|unk|> tea?'
  ```
* Complejidad: O(n) sobre longitud de la cadena.

---

### **Etapa 4 — Retorno**

```python
return text
```

* Devuelve la cadena reconstruida.
* El resultado mantiene coherencia gramatical básica, aunque puede incluir tokens `<|unk|>` que representan términos desconocidos.

**Salida final:**

```
'<|unk|>, do you <|unk|> tea?'
```

---

## **4) Análisis de robustez**

| Característica      | `V1`         | `V2`       | Efecto                    |        |           |     |                    |
| ------------------- | ------------ | ---------- | ------------------------- | ------ | --------- | --- | ------------------ |
| Manejo de OOV       | ❌            | ✅          | Evita `KeyError`          |        |           |     |                    |
| Tokens especiales   | ❌            | ✅ (`<      | unk                       | >`, `< | endoftext | >`) | Cobertura completa |
| Consistencia de IDs | ✅            | ✅          | Índices fijos             |        |           |     |                    |
| Formato de salida   | Básico       | Limpio     | Espacios corregidos       |        |           |     |                    |
| Uso en modelos      | Experimental | Entrenable | Compatible con embeddings |        |           |     |                    |

---

## **5) Complejidad global**

| Operación               | Complejidad temporal | Complejidad espacial |
| ----------------------- | -------------------- | -------------------- |
| Tokenización y limpieza | O(n)                 | O(n)                 |
| Verificación OOV        | O(n)                 | O(n)                 |
| Mapeo a IDs             | O(n)                 | O(n)                 |
| Decodificación          | O(n)                 | O(n)                 |

Donde *n* es el número de tokens procesados.

---

## **6) Importancia dentro del pipeline del LLM**

El `SimpleTokenizerV2` introduce el primer **componente de tolerancia semántica** en el preprocesamiento:

* Permite generalizar la codificación a entradas arbitrarias.
* Garantiza que el modelo pueda recibir texto sin fallar.
* Establece la base para el **vocabulario cerrado** usado en la pre-entrenamiento y fine-tuning de modelos autoregresivos tipo GPT.

---

**Conclusión:**
`SimpleTokenizerV2` convierte el prototipo V1 en un tokenizador **seguro y universal**, capaz de procesar cualquier texto de entrada sin excepciones, preservando el mapeo bidireccional entre texto e índices y respetando la estructura léxica requerida por los modelos de lenguaje basados en transformadores.


In [ ]:
text1 = 'Hello, do you like tea?'
text2 = 'In the sunlit terraces of the palace.'
text = ' <|endoftext|> '.join((text1, text2))
print(text)

Explicación exhaustiva, con análisis estructural, sintáctico y funcional:

---

### **1)**

```python
text1 = 'Hello, do you like tea?'
text2 = 'In the sunlit terraces of the palace.'
```

**Propósito:**
Definir dos fragmentos de texto (`str`) que simulan **entradas independientes** dentro de un corpus o dataset de entrenamiento.

**Detalles técnicos:**

* Ambos objetos son instancias del tipo `str` de Python, almacenadas como secuencias inmutables de *code points* Unicode.
* En memoria, Python usa una representación compacta (internamente UTF-8 o UCS-2/UCS-4 según compilación), y ambas cadenas residen en el heap con sus referencias gestionadas por el recolector de basura.
* El texto está formado por palabras, espacios y signos de puntuación (`','`, `'?'`, `'.'`), los cuales más adelante serán segmentados por el tokenizador.

**Semántica en el flujo del modelo:**

* Cada `textX` representa una unidad de entrada independiente (por ejemplo, un documento, diálogo o frase).
* En entrenamiento autoregresivo, se suelen concatenar múltiples secuencias con un **token delimitador** para permitir al modelo aprender los límites de contexto.

---

### **2)**

```python
text = ' <|endoftext|> '.join((text1, text2))
```

**Propósito:**
Concatenar las dos cadenas `text1` y `text2`, insertando entre ellas el **token especial de final de texto** `<|endoftext|>`.

---

#### **Desglose técnico:**

##### a) `' <|endoftext|> '`

* Es una cadena literal que contiene:

  * un espacio inicial `' '`
  * el token especial `<|endoftext|>`
  * un espacio final `' '`
* Este token actúa como **delimitador semántico** entre fragmentos, permitiendo distinguir el final de una secuencia y el inicio de otra dentro de un mismo batch textual.
* Este token fue añadido explícitamente al vocabulario en pasos anteriores.

##### b) `.join((text1, text2))`

* `.join(iterable)` concatena los elementos del iterable especificado, separándolos por la cadena sobre la que se invoca el método.
* El iterable aquí es una **tupla**: `(text1, text2)`.

  * Las tuplas son inmutables y de acceso indexado O(1).
* `join` recorre los elementos, los concatena, y **devuelve una nueva cadena**.

  * Implementación interna: crea un buffer de tamaño exacto mediante cálculo previo de longitud total → complejidad **O(n)** sobre la suma de longitudes.

##### c) Resultado exacto:

```
'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'
```

**Estructura conceptual resultante:**

```
[secuencia_1] + [token delimitador] + [secuencia_2]
```

Este patrón es típico en *datasets concatenados de texto plano*, donde el token `<|endoftext|>` reemplaza saltos de documento o marcadores de fin de muestra.

---

### **3)**

```python
print(text)
```

**Propósito:**
Mostrar en consola el resultado de la concatenación, verificando la correcta inserción del delimitador especial.

**Salida esperada:**

```
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
```

**Detalles técnicos:**

* `print()` llama internamente a `sys.stdout.write()` con una conversión implícita `str()` sobre su argumento.
* Añade un salto de línea final (`\n`) por defecto.
* Operación O(n) en la longitud total del texto.

---

### **Análisis conceptual**

| Elemento         | Función en la arquitectura del LLM        | Implicación                                                          |                                           |                                            |
| ---------------- | ----------------------------------------- | -------------------------------------------------------------------- | ----------------------------------------- | ------------------------------------------ |
| `text1`, `text2` | Fragmentos independientes del corpus      | Simulan secuencias separadas de entrenamiento                        |                                           |                                            |
| `<               | endoftext                                 | >`                                                                   | Token delimitador aprendido por el modelo | Enseña al modelo dónde termina una muestra |
| `.join()`        | Concatenación controlada                  | Permite construir datasets textuales continuos                       |                                           |                                            |
| Resultado final  | Texto continuo con marcador de separación | Base para el entrenamiento autoregresivo sin pérdida de segmentación |                                           |                                            |

---

### **Complejidad total**

| Operación             | Complejidad temporal             | Espacial        |
| --------------------- | -------------------------------- | --------------- |
| Creación de literales | O(1)                             | O(1)            |
| `.join()`             | O(n₁ + n₂ + k)` (longitud total) | O(n₁ + n₂ + k)` |
| `print()`             | O(n₁ + n₂ + k)`                  | O(1)            |

donde *n₁* y *n₂* son las longitudes de `text1` y `text2`, y *k* la longitud del delimitador `' <|endoftext|> '`.

---

### **Significado dentro del pipeline de un LLM**

Este paso demuestra la **integración práctica de tokens especiales** dentro del flujo textual.
En los modelos tipo GPT:

* `<|endoftext|>` marca la **frontera de contexto** en el corpus concatenado.
* Durante la inferencia, el modelo puede generar este token para indicar que debe **detener la producción de texto**.
* Durante el entrenamiento, el modelo aprende que la probabilidad de este token aumenta cuando una muestra llega a su fin.

En resumen:

> Este fragmento construye una cadena compuesta que respeta la semántica de fin de texto, asegurando continuidad sintáctica y segmentación explícita entre unidades lingüísticas —un paso esencial en la preparación de corpora para el entrenamiento autoregresivo de un LLM.


In [ ]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

Explicación exhaustiva del comportamiento de este bloque, tanto en nivel de ejecución como en su papel dentro del flujo del modelo de lenguaje:

---

### **1)**

```python
tokenizer = SimpleTokenizerV2(vocab)
```

**Propósito:**
Crear una instancia del tokenizador robusto (`SimpleTokenizerV2`) empleando el vocabulario extendido que incluye los tokens especiales `<|endoftext|>` y `<|unk|>`.

---

#### **Comportamiento interno:**

* Se ejecuta el **constructor** `__init__()` definido en la clase:

  ```python
  def __init__(self, vocab):
      self.str_to_int = vocab
      self.int_to_str = {i: s for s, i in vocab.items()}
  ```

* **Estructuras creadas:**

  1. `self.str_to_int`: diccionario que mapea **token → ID entero**

     * Ejemplo:

       ```
       {',': 3, '?': 9, 'Hello': 240, '<|endoftext|>': 1127, '<|unk|>': 1128}
       ```
  2. `self.int_to_str`: diccionario inverso **ID → token**, generado mediante comprensión:

     ```python
     {0: '!', 1: '"', ..., 1127: '<|endoftext|>', 1128: '<|unk|>'}
     ```

* **Complejidad:**

  * Construcción: O(n) (recorre el vocabulario completo).
  * Accesos posteriores: O(1) promedio (hash table).

* **Resultado:**
  Un tokenizador funcional capaz de codificar y decodificar cualquier texto sin errores de clave.

---

### **2)**

```python
print(tokenizer.encode(text))
```

**Propósito:**
Aplicar el proceso de **tokenización y codificación numérica** sobre la cadena `text` que contiene el delimitador `<|endoftext|>` y mostrar el resultado.

---

#### **Flujo interno dentro de `encode()`**

```python
def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids
```

---

### **Etapa 1 — Tokenización regular**

```python
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
```

**Entrada:**

```
text = 'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'
```

**Patrón:**

* `[,.?_!"()']` → divide por puntuación básica.
* `|--` → captura doble guion literal.
* `|\s` → divide por espacios.

**Resultado inicial (lista con separadores y vacíos):**

```
['Hello', ',', '', ' ', 'do', ' ', 'you', ' ', 'like', ' ', 'tea', '?', ' ', '<|endoftext|>', ' ', 'In', ' ', 'the', ' ', 'sunlit', ' ', 'terraces', ' ', 'of', ' ', 'the', ' ', 'palace', '.', '']
```

**Complejidad:** O(n) sobre número de caracteres.

---

### **Etapa 2 — Limpieza**

```python
preprocessed = [item.strip() for item in preprocessed if item.strip()]
```

* `strip()` elimina espacios al inicio y fin.
* `if item.strip()` descarta vacíos.
* Resultado limpio:

  ```
  ['Hello', ',', 'do', 'you', 'like', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
  ```

**Complejidad:** O(m), donde *m* = número de tokens detectados.

---

### **Etapa 3 — Sustitución de palabras desconocidas**

```python
preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
```

**Proceso:**

* Comprueba si cada token está en `self.str_to_int`.
* Si no, lo reemplaza por `<|unk|>`.

**Caso particular:**

* `"Hello"` y `"like"` probablemente **no existen** en el vocabulario del cuento *The Verdict*.
* Por tanto, serán reemplazados por `<|unk|>`.

**Resultado tras esta etapa:**

```
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
```

**Importante:**
El token `<|endoftext|>` **sí existe** en el vocabulario, por lo que se conserva literalmente.

---

### **Etapa 4 — Mapeo token → ID**

```python
ids = [self.str_to_int[s] for s in preprocessed]
```

* Convierte cada token textual en su identificador numérico.
* Acceso a diccionario: O(1) promedio por token.
* Complejidad total: O(m).

**Ejemplo de salida hipotética:**

```
[1128, 3, 42, 57, 1128, 89, 9, 1127, 215, 27, 390, 420, 84, 27, 500, 7]
```

donde:

| Token    | ID (ejemplo) |    |      |
| -------- | ------------ | -- | ---- |
| `<       | unk          | >` | 1128 |
| `,`      | 3            |    |      |
| `do`     | 42           |    |      |
| `you`    | 57           |    |      |
| `tea`    | 89           |    |      |
| `?`      | 9            |    |      |
| `<       | endoftext    | >` | 1127 |
| `In`     | 215          |    |      |
| `the`    | 27           |    |      |
| `palace` | 500          |    |      |
| `.`      | 7            |    |      |

---

### **Etapa 5 — Retorno y visualización**

```python
return ids
```

* Devuelve la lista de IDs enteros.
* `print()` muestra el resultado en consola como lista Python estándar.

**Salida visual esperada:**

```
[1128, 3, 42, 57, 1128, 89, 9, 1127, 215, 27, 390, 420, 84, 27, 500, 7]
```

*(Los valores dependen del orden real del vocabulario.)*

---

## **Análisis conceptual**

| Elemento   | Significado funcional        | Papel en el LLM                               |                                                          |                                               |
| ---------- | ---------------------------- | --------------------------------------------- | -------------------------------------------------------- | --------------------------------------------- |
| `<         | unk                          | >`                                            | Token de sustitución para palabras fuera del vocabulario | Permite codificar cualquier texto sin errores |
| `<         | endoftext                    | >`                                            | Delimitador explícito entre secuencias                   | Enseña al modelo los límites contextuales     |
| `encode()` | Conversión texto → índices   | Base para construir tensores de entrenamiento |                                                          |                                               |
| `vocab`    | Espacio discreto de símbolos | Define el universo léxico del modelo          |                                                          |                                               |

---

### **Complejidad global del método**

| Etapa                | Operación        | Complejidad temporal | Espacial |
| -------------------- | ---------------- | -------------------- | -------- |
| Tokenización (regex) | `re.split()`     | O(n)                 | O(n)     |
| Limpieza             | `strip` + filtro | O(n)                 | O(n)     |
| Sustitución OOV      | Búsqueda hash    | O(n)                 | O(n)     |
| Mapeo a IDs          | Diccionario hash | O(n)                 | O(n)     |

Donde *n* es el número de caracteres o tokens procesados.

---

### **Interpretación dentro del pipeline del LLM**

Este paso completa la fase de **codificación simbólica robusta**:

1. Garantiza que cualquier texto pueda transformarse en una secuencia de enteros, eliminando errores en la fase de batch tokenization.
2. Mantiene los límites entre documentos con `<|endoftext|>`.
3. Proporciona una representación **determinista y reproducible**, indispensable para entrenar embeddings o modelos de atención.

---

**Conclusión:**
La instrucción

```python
print(tokenizer.encode(text))
```

produce una secuencia de IDs enteros que representa de forma numérica la cadena

```
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
```

con manejo seguro de tokens desconocidos mediante `<|unk|>` y preservación del token `<|endoftext|>`.
Este es el paso en que el texto humano se convierte, por primera vez, en una estructura formal apta para el aprendizaje en un modelo de lenguaje.


In [ ]:
print(tokenizer.decode(tokenizer.encode(text)))

Explicación exhaustiva, incluyendo funcionamiento interno completo, estructura de datos implicadas y relevancia dentro del flujo de un LLM:

---

## **1)**

```python
print(tokenizer.decode(tokenizer.encode(text)))
```

**Propósito:**
Ejecutar el ciclo completo **texto → tokens → IDs → texto**, validando que el tokenizador `SimpleTokenizerV2` sea **reversible**, consistente y tolerante a palabras desconocidas (*out-of-vocabulary*, OOV).

---

## **2) Flujo general de ejecución**

La expresión se evalúa de adentro hacia afuera:

1. `tokenizer.encode(text)`
   → convierte el texto crudo en una lista de identificadores enteros.
2. `tokenizer.decode(...)`
   → traduce esos identificadores de vuelta a texto.
3. `print(...)`
   → muestra la cadena reconstruida resultante.

---

## **3) Entrada inicial**

```python
text = 'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'
```

El texto contiene:

* palabras comunes (`do`, `you`, `tea`, `the`, etc.),
* palabras potencialmente desconocidas (`Hello`, `like`),
* un token especial `<|endoftext|>` que **sí está en el vocabulario**,
* puntuación (`?`, `,`, `.`).

---

## **4) Etapas internas de `encode()`**

El método `encode()` ejecuta:

```python
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
ids = [self.str_to_int[s] for s in preprocessed]
return ids
```

### **Etapa 4.1 — Tokenización**

Divide el texto en palabras, signos y espacios, conservando separadores.

**Salida intermedia:**

```
['Hello', ',', 'do', 'you', 'like', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
```

### **Etapa 4.2 — Sustitución de palabras fuera del vocabulario**

* `"Hello"` y `"like"` no están en el vocabulario del cuento *The Verdict*.
* Se reemplazan por `<|unk|>` (token desconocido).

**Resultado limpio:**

```
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
```

### **Etapa 4.3 — Conversión a IDs**

Cada token se transforma en su índice entero del vocabulario (`self.str_to_int`).

**Ejemplo conceptual:**

```
[1128, 3, 42, 57, 1128, 89, 9, 1127, 215, 27, 390, 420, 84, 27, 500, 7]
```

---

## **5) Etapas internas de `decode()`**

El método `decode()`:

```python
def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text
```

### **Etapa 5.1 — Reconstrucción desde IDs**

Convierte cada entero a su token textual original con `self.int_to_str[i]`.

**Salida intermedia (lista de tokens):**

```
['<|unk|>', ',', 'do', 'you', '<|unk|>', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']
```

### **Etapa 5.2 — Unión con espacios**

```python
" ".join([...])
```

Concatena tokens con un espacio entre ellos:

```
'<|unk|> , do you <|unk|> tea ? <|endoftext|> In the sunlit terraces of the palace .'
```

### **Etapa 5.3 — Limpieza tipográfica**

```python
re.sub(r'\s+([,.?!"()\'])', r'\1', text)
```

* Busca cualquier espacio antes de signos de puntuación y los elimina.
* No altera los tokens `<|unk|>` ni `<|endoftext|>`, porque no coinciden con el patrón de signos.

**Resultado limpio:**

```
'<|unk|>, do you <|unk|> tea? <|endoftext|> In the sunlit terraces of the palace.'
```

---

## **6) Etapa final — Impresión**

```python
print(...)
```

Muestra la cadena final resultante:

```
<|unk|>, do you <|unk|> tea? <|endoftext|> In the sunlit terraces of the palace.
```

---

## **7) Análisis conceptual del resultado**

| Elemento    | Significado                                                 | Función en el flujo                    |                                                                    |                                               |
| ----------- | ----------------------------------------------------------- | -------------------------------------- | ------------------------------------------------------------------ | --------------------------------------------- |
| `<          | unk                                                         | >`                                     | Representa tokens no presentes en el vocabulario (“Hello”, “like”) | Permite codificar sin error cualquier entrada |
| `,` `?` `.` | Signos de puntuación reconstruidos sin espacios incorrectos | Verifica consistencia del reensamblado |                                                                    |                                               |
| `<          | endoftext                                                   | >`                                     | Delimitador explícito entre fragmentos textuales                   | Enseña límites de contexto al modelo          |

---

## **8) Propiedades técnicas del proceso completo**

| Propiedad                           | Descripción                                                                        | Consecuencia                               |    |                                             |
| ----------------------------------- | ---------------------------------------------------------------------------------- | ------------------------------------------ | -- | ------------------------------------------- |
| **Reversibilidad parcial**          | Las palabras conocidas se reconstruyen idénticas; las desconocidas vuelven como `< | unk                                        | >` | Pérdida controlada de información semántica |
| **Determinismo**                    | La codificación y decodificación son funciones puras, sin aleatoriedad             | Resultados reproducibles                   |    |                                             |
| **Consistencia del espacio léxico** | Solo aparecen tokens del vocabulario                                               | Compatible con entrenamiento de embeddings |    |                                             |
| **Complejidad total**               | O(n) en número de tokens, tanto en encode como en decode                           | Escalable para corpus grandes              |    |                                             |

---

## **9) Significado dentro del pipeline del LLM**

Este paso valida la **coherencia bidireccional del tokenizador**, condición necesaria para:

* entrenar embeddings consistentes (`token_id → vector → token_id`);
* garantizar que cada ID corresponde a un símbolo textual válido;
* controlar el flujo de fin de secuencia mediante `<|endoftext|>`;
* permitir inferencia segura con textos arbitrarios (manejo OOV).

En entrenamiento real, este mecanismo garantiza que:

> Cualquier texto del dataset puede ser mapeado a secuencias numéricas válidas y luego reconstruido de manera legible, asegurando estabilidad en la codificación semántica del modelo.

---

## **10) Resultado final**

**Salida esperada en consola:**

```
<|unk|>, do you <|unk|> tea? <|endoftext|> In the sunlit terraces of the palace.
```

**Interpretación:**

* El tokenizador reemplazó “Hello” y “like” por `<|unk|>`.
* Preservó puntuación, estructura y delimitador `<|endoftext|>`.
* El ciclo `encode → decode` demuestra que el tokenizador es **robusto, determinista y seguro frente a OOV**, cumpliendo con los requisitos de un preprocesador de texto para un LLM autoregresivo.


# 2.5 **Byte pair encoding**
# 2.5 **Codificación de pares de bytes**

In [ ]:
pip install tiktoken

Esa celda instala la librería **`tiktoken`**, que es el tokenizador oficial de OpenAI.

**Explicación funcional:**

* `pip` es el gestor de paquetes de Python.
* `install` descarga e instala el paquete desde el repositorio de PyPI.
* `tiktoken` permite convertir texto en tokens (números) del mismo modo que lo hacen modelos como GPT-2 o GPT-3.

**Resultado esperado:**
Al ejecutarse, tu entorno (Colab) mostrará mensajes de instalación y luego podrás usarlo con:

```python
import tiktoken
```

**Propósito dentro del notebook:**
Preparar el entorno para tokenizar texto antes de crear datasets y entrenar el modelo en los pasos siguientes.


In [ ]:
from importlib.metadata import version
import tiktoken
print('tiktoken version: ', version('tiktoken'))

**Explicación funcional de la celda:**

1. `from importlib.metadata import version`
   — Importa una función del módulo estándar de Python que permite consultar la versión instalada de un paquete.

2. `import tiktoken`
   — Carga la librería `tiktoken` previamente instalada, necesaria para tokenizar texto.

3. `print('tiktoken version: ', version('tiktoken'))`
   — Imprime en pantalla la versión exacta del paquete `tiktoken` que está activa en el entorno de ejecución.

**Propósito:**
Verificar que la instalación se realizó correctamente y conocer qué versión de `tiktoken` se usará en el notebook.

**Entrada:** ninguna.
**Salida:** una línea de texto como

```
tiktoken version:  0.7.0
```

(según la versión instalada en tu entorno).


In [12]:
tokenizer = tiktoken.get_encoding('gpt2')

text = 'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the someunknowPlace.'

integers = tokenizer.encode(text, allowed_special={'<|endoftext|>'})

print(integers)

NameError: name 'tiktoken' is not defined

Explicación línea por línea. Precisa y técnica.

1. `tokenizer = tiktoken.get_encoding('gpt2')`

* Qué hace: carga un objeto `Encoding` predefinido con el vocabulario y merges BPE del tokenizador de GPT-2.
* Firma relevante: `tiktoken.get_encoding(name: str) -> Encoding`.
* Parámetros:

  * `name` (obligatorio): cadena con el nombre del *encoding*. Ejemplos comunes: `'gpt2'`, `'p50k_base'`, `'cl100k_base'`.
* Opcionales: esta función no expone parámetros opcionales.
* Notas: para mapear por modelo se usa `tiktoken.encoding_for_model(model_name)`; aquí se fuerza explícitamente el esquema GPT-2.

2.

```python
text = 'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the someunknowPlace.'
```

* Qué hace: define la cadena de entrada a tokenizar.
* Contenido relevante: incluye el *special token* de GPT-2 `"<|endoftext|>"` (ID estándar 50256 en GPT-2) incrustado en el texto.

3.

```python
integers = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
```

* Qué hace: tokeniza `text` y devuelve una lista de IDs enteros (`list[int]`) que representan los tokens BPE, incluyendo el *special token* permitido.
* Firma relevante (tiktoken):

  * `Encoding.encode(text: str, *, allowed_special: Union[Literal['all'], Set[str]] = set(), disallowed_special: Union[Literal['all'], Set[str]] = 'all') -> List[int]`
* Parámetros:

  * `text` (obligatorio): cadena a tokenizar.
  * `allowed_special` (opcional, *keyword-only*): conjunto de *special tokens* que se aceptan en el texto tal cual. Aquí se pasa `{'<|endoftext|>'}` para que esa secuencia se reconozca como un único token especial y no como texto normal. Puede ser también `'all'` para permitir todos los especiales o `set()` para no permitir ninguno.
  * `disallowed_special` (opcional, *keyword-only*; por defecto `'all'`): define qué *special tokens* están prohibidos. Si `'all'`, lanzará error si aparece algún especial no listado en `allowed_special`. Si se pasa `set()`, no prohibe ninguno adicional.
* Resultado: `integers` es una lista de IDs en el rango del vocabulario GPT-2. En la posición donde aparece `<|endoftext|>` se incluirá su ID especial (50256) en la secuencia.

4. `print(integers)`

* Qué hace: imprime la lista de IDs de tokens resultante, p. ej. `[15496, 11, 360, ... , 50256, ...]`
* Observación: la lista exacta depende del BPE de GPT-2. Palabras compuestas como `"someunknowPlace"` se dividirán en subpalabras según las merges del vocabulario.

Notas prácticas

* Si quieres ignorar cualquier *special token* escrito en el texto y tratarlos como caracteres normales, usa `tokenizer.encode(text)` con `allowed_special=set()` (por defecto) o `tokenizer.encode_ordinary(text)` que nunca reconoce especiales.
* Para lotes: `tokenizer.encode_batch(list_of_texts, allowed_special=...)`.
* Para decodificar: `tokenizer.decode(integers)` o `tokenizer.decode_bytes(...)` según el caso.


In [ ]:
strings = tokenizer.decode(integers)
print(strings)


**Explicación funcional de la celda:**

1. `strings = tokenizer.decode(integers)`

   * Usa el método `decode` del objeto `tokenizer` (de `tiktoken`) para convertir una lista de **tokens numéricos** (`integers`) de vuelta a **texto legible**.
   * Cada número en `integers` representa una palabra, sub-palabra o símbolo según el vocabulario del tokenizador.
   * El método reconstruye el texto original uniendo esos fragmentos.

2. `print(strings)`

   * Muestra en pantalla el texto reconstruido.

**Propósito:**
Comprobar que la conversión *tokens → texto* funciona correctamente y que los números representan el texto esperado.

**Entradas:**

* `tokenizer`: instancia de `tiktoken.Encoding` creada antes.
* `integers`: lista de IDs de tokens (por ejemplo `[464, 3290, 318]`).

**Salida:**

* `strings`: cadena de texto equivalente (por ejemplo `"Hello world!"`).

**Uso dentro del flujo:**
Se utiliza para verificar la relación entre el texto original, los tokens numéricos generados y su decodificación, paso previo a la creación del dataset de entrenamiento.


In [ ]:
for token_id in integers:

  token_text = tokenizer.decode([token_id])

  print(f'{token_id} ----> {token_text}')

# Explicación de la celda de código:

```python
for token_id in integers:

  token_text = tokenizer.decode([token_id])

  print(f'{token_id} ----> {token_text}')
```

### Línea 1

```python
for token_id in integers:
```

* Qué hace: inicia un bucle `for` que itera sobre cada elemento en la lista `integers`.
* `integers` previamente contiene una lista de IDs de tokens (enteros) generados por `tokenizer.encode(...)`.
* En cada iteración, la variable `token_id` toma uno de los valores de esa lista.
* Sin parámetros adicionales.
* Propósito: recorrer cada token individual para inspeccionarlo o procesarlo por separado.

### Línea 2

```python
  token_text = tokenizer.decode([token_id])
```

* Qué hace: llama al método `decode` del objeto `tokenizer` (que es un objeto `Encoding` de tiktoken) para convertir un *listado* que contiene un único token ID en su representación de texto (cadena).
* Firma aproximada: `Encoding.decode(ids: list[int], *, disallowed_special: … maybe) -> str` (según documentación y ejemplos).
* Parámetros:

  * `ids`: lista de enteros, aquí se pasa `[token_id]` (lista de un solo elemento).
  * Otros parámetros opcionales: en versiones recientes puede haber opciones como `truncate_at_eos` u otros ajustes para especiales. Por ejemplo, en un wrapper de PyTorch se ve `decode(token_ids: List[int], truncate_at_eos: bool = True)` ([docs.pytorch.org][1]).
* Resultado: `token_text` es la cadena de texto correspondiente al token ID. Puede incluir espacios iniciales o caracteres especiales según la tokenización BPE del modelo.
* Notas de actualización: según documentación oficial, `Encoding.decode(...)` devolverá el texto original al agrupar una lista completa de tokens. Decodificar token por token (como aquí) puede funcionar, pero puede que algunos tokens no se decodifiquen aislados “perfectamente” debido a que algunos sub-tokens dependen del contexto (ver issue #202: decodificación de símbolos matemáticos puede fallar para `decode_single_token_bytes`). ([GitHub][2])
* Propósito: ver qué texto corresponde a cada ID individual, útil para inspección, depuración o entender cómo el tokenizador segmenta el texto.

### Línea 3

```python
  print(f'{token_id} ----> {token_text}')
```

* Qué hace: imprime una línea formateada que muestra el token ID y la cadena `token_text` a la que corresponde.
* No hay función aparte con parámetros especiales. El f-string `f'{token_id} ----> {token_text}'` sustituye los valores correspondientes.
* Propósito: visualización clara del mapeo “ID → texto” para cada token.

### Consideraciones generales

* El bucle produce, para cada token en la secuencia, una línea como:

  ```
  15496 ----> Hello
  11 ----> ,
  50256 ----> <|endoftext|>
  …
  ```

  (ejemplo de posibles valores)
* El uso de `decode([token_id])` por token individual es educativo, pero no es lo más eficiente para decodificar una frase completa (mejor pasar la lista completa de IDs a `decode`).
* Algunos tokens pueden contener espacios al inicio o finales porque el esquema BPE de GPT-2 incluye espacios como parte del token, por ejemplo `" world"` vs `"world"`.
* Es importante que `tokenizer` sea del mismo encoding que se usó para `encode`, de lo contrario habrá inconsistencias.

### En el contexto del capítulo 2 de tu estudio

* Esta celda te ayuda a entender **cómo** cada token ID se traduce de vuelta a texto legible y así ver **qué porciones de texto** fueron asignadas a cada ID.
* Te da visibilidad del comportamiento interno del tokenizador: la segmentación, los espacios, los *special tokens*, etc.
* Permite detectar casos especiales (como el token `<|endoftext|>`) y ver cómo se representa internamente.


**Explicación detallada de la celda (con énfasis en los parámetros de las funciones):**

```python
for token_id in integers:
    token_text = tokenizer.decode([token_id])
    print(f'{token_id} ----> {token_text}')
```

---

### 1. Qué hace el bloque en general

Recorre una lista de tokens numéricos (`integers`) y muestra la relación **ID → texto** de cada uno.
Sirve para visualizar cómo el tokenizador representa cada fragmento del texto con un número.

---

### 2. Desglose línea a línea

#### `for token_id in integers:`

* **Entrada:** `integers` es una lista de enteros, cada uno corresponde a un *token ID*.
  Ejemplo: `[464, 3290, 318, 262]`.
* **Salida en cada iteración:** `token_id` toma uno de esos valores, por ejemplo `464`.
* **Propósito:** iterar por cada número y decodificarlo por separado.

---

#### `token_text = tokenizer.decode([token_id])`

* **Método:** `decode()` pertenece al objeto `tokenizer`, una instancia de `tiktoken.Encoding`.
* **Parámetro recibido:**

  * Espera una **lista o secuencia de IDs enteros** que representan tokens (`list[int]` o `Iterable[int]`).
  * En este caso, se pasa **una lista con un solo elemento**: `[token_id]`.
    → Es importante que sea lista, no un entero, porque el método espera un *conjunto de tokens*, no un solo número.
* **Acción interna:** convierte la secuencia de IDs en texto legible, uniendo los fragmentos según las reglas del vocabulario interno de `tiktoken`.
* **Valor devuelto:** una cadena (`str`) que representa el texto equivalente a ese token.
  Ejemplo:

  ```
  token_id = 464
  token_text = 'Hello'
  ```

---

#### `print(f'{token_id} ----> {token_text}')`

* **Entradas:**

  * `token_id`: el número del token.
  * `token_text`: el texto correspondiente.
* **Salida:** imprime la relación número → palabra, por ejemplo:

  ```
  464 ----> Hello
  3290 ----> world
  318 ----> !
  ```

---

### 3. Resumen de parámetros y retornos del método `decode`

| Elemento                   | Tipo esperado   | Descripción                          | Tipo devuelto            |
| -------------------------- | --------------- | ------------------------------------ | ------------------------ |
| `tokenizer.decode(tokens)` | `Iterable[int]` | Lista de IDs de tokens a decodificar | `str` (texto resultante) |

---

### 4. Propósito práctico

Este fragmento sirve para:

* Entender cómo cada número (token) se traduce a texto.
* Visualizar cómo el modelo ve el texto internamente, dividido en unidades numéricas.
* Comprobar que la tokenización y decodificación funcionan correctamente antes de entrenar.

---

¿Quieres que use este mismo formato para las siguientes celdas que vayas enviando (con la tabla de parámetros y retornos incluida)?


# EXERCISE 2.1 BYTE PAIR ENCODING OF UNKNOWN WORDS
# EJERCICIO 2.1 CODIFICACIÓN DE PARES DE BYTES DE PALABRAS DESCONOCIDAS

Usa el tokenizador BPE de la biblioteca tiktoken sobre las palabras desconocidas «Akwirw ier» e imprime los IDs de los tokens individuales. Luego, llama a la función decode para cada uno de los enteros resultantes en esta lista para reproducir el mapeo mostrado en la Figura 2.11. Por último, llama al método decode con la lista de IDs de los tokens para verificar si puede reconstruir la entrada original, «Akwirw ier».

In [13]:
text_akw = 'Akwirw ier'
integers_akw = tokenizer.encode(text_akw)
print(integers_akw)

KeyError: 'Akwirw'

**Explicación de la celda de código**

```python
text_akw = 'Akwirw ier'
integers_akw = tokenizer.encode(text_akw)
print(integers_akw)
```

### Línea 1

```python
text_akw = 'Akwirw ier'
```

* Asigna a la variable `text_akw` una cadena de texto con valor `'Akwirw ier'`.
* Esta cadena será el texto que se desea tokenizar.
* No interviene ninguna función ni clase; es solo la creación de un literal de tipo `str`.

### Línea 2

```python
integers_akw = tokenizer.encode(text_akw)
```

* Llama al método `encode` del objeto `tokenizer` (instancia de `tiktoken.Encoding`).
* **Firma relevante:**

  ```python
  Encoding.encode(
      text: str,
      *,
      allowed_special: Union[Literal['all'], Set[str]] = set(),
      disallowed_special: Union[Literal['all'], Set[str]] = 'all'
  ) -> List[int]
  ```
* **Parámetros:**

  * `text` (obligatorio): la cadena que se tokeniza (`'Akwirw ier'`).
  * `allowed_special` (opcional, por defecto `set()`): conjunto de *special tokens* permitidos en el texto. Aquí no se pasa, por lo tanto, no se permite ninguno.
  * `disallowed_special` (opcional, por defecto `'all'`): indica qué *special tokens* no se permiten. Al ser `'all'`, lanzaría error si aparece uno no permitido.
* **Comportamiento:**
  Divide el texto en subunidades llamadas *tokens* según las reglas del vocabulario BPE del modelo (en este caso, GPT-2).
  Cada token se convierte en un entero correspondiente a su índice en el vocabulario.
  Como `'Akwirw ier'` probablemente no existe en el vocabulario GPT-2, se dividirá en sub-tokens más pequeños o incluso en caracteres sueltos.
* **Salida:**
  Una lista de enteros (`List[int]`), por ejemplo: `[32xxx, 502xx, ...]`, dependiendo del vocabulario usado.
  Esta lista se almacena en la variable `integers_akw`.

### Línea 3

```python
print(integers_akw)
```

* Imprime la lista de IDs de tokens resultante de la tokenización.
* Permite visualizar cómo el modelo segmentó el texto y qué IDs asignó.


In [14]:
for token_id in integers_akw:

  token_text = tokenizer.decode([token_id])

  print(f'{token_id} ---> {token_text}')

NameError: name 'integers_akw' is not defined

**Línea 1**

```python
for token_id in integers_akw:
```

* **Qué hace:** inicia un bucle `for` que recorre secuencialmente todos los elementos de la lista `integers_akw`.
* **Contexto:** `integers_akw` contiene los identificadores numéricos de los tokens generados al codificar el texto con el tokenizador GPT-2.
* **Comportamiento:** en cada iteración, la variable `token_id` toma el valor de un ID de token.
* **Uso:** permite procesar o inspeccionar cada token individualmente.

**Línea 2**

```python
token_text = tokenizer.decode([token_id])
```

* **Qué hace:** convierte el identificador numérico del token en su representación textual.
* **Firma del método:**
  `Encoding.decode(tokens: list[int], *, disallowed_special: Union[Literal['all'], Set[str]] = set()) -> str`
* **Parámetros:**

  * `tokens`: lista de enteros que representan los IDs de los tokens a decodificar.
  * `disallowed_special` (opcional): conjunto de *special tokens* que no deben decodificarse; por defecto está vacío, por lo que todos se permiten.
* **Comportamiento interno:** el método consulta el vocabulario del tokenizador y reconstruye el fragmento de texto correspondiente.
* **Salida:** cadena (`str`) con el texto asociado al token.
* **Nota:** algunos tokens incluyen espacios iniciales porque el esquema de GPT-2 incorpora el espacio como parte del token para optimizar la segmentación.

**Línea 3**

```python
print(f'{token_id} ---> {token_text}')
```

* **Qué hace:** imprime el par formado por el identificador numérico y su texto correspondiente.
* **Mecanismo:** la cadena f-string inserta los valores de las variables dentro del texto mostrado.
* **Salida típica:**

  ```
  1234 ---> A
  5678 ---> kw
  9123 ---> ir
  ```
* **Propósito:** visualizar cómo el tokenizador segmenta el texto y qué fragmento representa cada token en el vocabulario.


In [ ]:
strings_akw = tokenizer.decode(integers_akw)
print(strings_akw)

**Línea 1**

```python
strings_akw = tokenizer.decode(integers_akw)
```

* **Qué hace:** convierte toda la secuencia de IDs de tokens almacenada en `integers_akw` de vuelta a texto legible.
* **Firma del método:**
  `Encoding.decode(tokens: list[int], *, disallowed_special: Union[Literal['all'], Set[str]] = set()) -> str`
* **Parámetros:**

  * `tokens`: lista completa de enteros que representan los IDs de los tokens a decodificar.
  * `disallowed_special` (opcional): conjunto de *special tokens* que no deben decodificarse; por defecto está vacío, por lo que todos los tokens conocidos se convierten a texto.
* **Comportamiento interno:** toma la lista completa de tokens, concatena las cadenas asociadas a cada ID y devuelve una reconstrucción aproximada del texto original.
* **Salida:** una cadena (`str`) con el texto resultante, almacenada en `strings_akw`.
* **Nota:** si el texto contenía tokens especiales, estos se representarán con sus marcadores (`<|endoftext|>`) salvo que se configuren como no permitidos.

**Línea 2**

```python
print(strings_akw)
```

* **Qué hace:** muestra en la salida estándar la cadena reconstruida.
* **Propósito:** verificar que la decodificación de todos los tokens reproduce correctamente el texto original, confirmando la correspondencia entre los procesos de *encode* y *decode*.


# **2.6 Data sampling with a sliding window**

In [ ]:
with open('./sample_data/llm_from_scratch/the-verdict.txt', 'r', encoding='utf-8') as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

**Línea 1**

```python
with open('./sample_data/llm_from_scratch/the-verdict.txt', 'r', encoding='utf-8') as f:
```

* **Qué hace:** abre el archivo de texto `the-verdict.txt` en modo lectura (`'r'`) y con codificación UTF-8.
* **Firma relevante:** `open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)`
* **Parámetros:**

  * `file`: ruta del archivo a abrir (`'./sample_data/llm_from_scratch/the-verdict.txt'`).
  * `mode`: `'r'` indica que se abrirá solo para lectura.
  * `encoding`: `'utf-8'` especifica el formato de codificación del texto.
  * Los demás parámetros son opcionales (por defecto no se modifican).
* **Comportamiento:** crea un *context manager* que asigna el manejador de archivo a la variable `f`.
* **Ventaja:** el bloque `with` garantiza que el archivo se cierre automáticamente al terminar su uso.

**Línea 2**

```python
  raw_text = f.read()
```

* **Qué hace:** lee todo el contenido del archivo abierto y lo almacena en la variable `raw_text`.
* **Firma:** `file.read(size=-1) -> str`
* **Parámetros:**

  * `size` (opcional, por defecto `-1`): número de caracteres a leer; `-1` indica que se lea todo el archivo.
* **Salida:** una cadena de texto con el contenido completo del archivo.
* **Resultado:** `raw_text` contiene el texto que luego será tokenizado.

**Línea 3**

```python
enc_text = tokenizer.encode(raw_text)
```

* **Qué hace:** convierte el texto leído (`raw_text`) en una lista de IDs de tokens según el esquema del tokenizador GPT-2.
* **Firma:**
  `Encoding.encode(text: str, *, allowed_special=set(), disallowed_special='all') -> List[int]`
* **Parámetros:**

  * `text`: cadena a tokenizar.
  * `allowed_special` y `disallowed_special`: controlan la admisión de *special tokens* (se usan los valores por defecto).
* **Salida:** lista de enteros (`List[int]`) que representan los tokens del archivo.
* **Resultado:** `enc_text` contiene la secuencia codificada que el modelo usaría como entrada.

**Línea 4**

```python
print(len(enc_text))
```

* **Qué hace:** imprime la longitud de la lista `enc_text`.
* **Salida:** número total de tokens generados a partir del texto del archivo.
* **Propósito:** medir el tamaño del conjunto de datos en términos de tokens, lo cual es fundamental para planificar entrenamiento y memoria.


In [ ]:
enc_sample = enc_text[50:]
print(len(enc_sample))


**Línea 1**

```python
enc_sample = enc_text[50:]
```

* **Qué hace:** crea una nueva lista llamada `enc_sample` que contiene una porción de `enc_text` a partir del índice 50 hasta el final.
* **Sintaxis:** `lista[inicio:fin]` realiza un *slicing* (sublista) en Python.
* **Parámetros implícitos:**

  * `inicio = 50`: indica que la nueva lista comienza en el elemento de índice 50.
  * `fin` no se especifica, por lo que toma todos los elementos hasta el último.
* **Comportamiento:** copia los tokens desde la posición 50 en adelante, sin modificar la lista original.
* **Salida:** una nueva lista `enc_sample` que contiene `len(enc_text) - 50` elementos.
* **Uso:** permite trabajar con un subconjunto del texto tokenizado, por ejemplo para pruebas o muestreo.

**Línea 2**

```python
print(len(enc_sample))
```

* **Qué hace:** imprime la longitud de la lista `enc_sample`.
* **Resultado:** muestra el número total de tokens contenidos en esa muestra parcial.
* **Propósito:** verificar que el *slicing* se aplicó correctamente y conocer cuántos tokens quedaron tras eliminar los primeros 50.


In [15]:
context_size = 4 # esta ventana de contexto determina cuantos tokens son inluídos en la entrada
x = enc_sample[:context_size]

y = enc_sample[1:context_size + 1]

print(f'x: {x}')
print(f'y: {y}')

NameError: name 'enc_sample' is not defined

**Línea 1**

```python
context_size = 4
```

* **Qué hace:** define la variable `context_size` con el valor entero `4`.
* **Propósito:** establece el tamaño del contexto o ventana de tokens que se usará para construir pares de entrada–salida en el entrenamiento del modelo.
* En este caso, se tomarán secuencias de 4 tokens consecutivos como muestra de entrada.

**Línea 2**

```python
x = enc_sample[:context_size]
```

* **Qué hace:** selecciona los primeros `context_size` (4) elementos de la lista `enc_sample`.
* **Sintaxis:** `lista[:n]` devuelve los elementos desde el índice `0` hasta `n-1`.
* **Salida:** lista `x` que contiene los primeros 4 tokens de `enc_sample`.
* **Uso:** representa el contexto de entrada para la predicción del siguiente token.

**Línea 3**

```python
y = enc_sample[1:context_size + 1]
```

* **Qué hace:** toma una porción de `enc_sample` desplazada un elemento hacia adelante.
* **Parámetros implícitos:**

  * Inicio en el índice `1`.
  * Fin en `context_size + 1` (es decir, `5`).
* **Salida:** lista `y` que contiene los tokens que siguen inmediatamente a los de `x`.
* **Uso:** representa los tokens objetivo (*targets*) que el modelo debe predecir dados los contextos `x`.
* **Relación:** cada elemento `y[i]` es el “siguiente token” de `x[i]`.

**Línea 4**

```python
print(f'x: {x}')
```

* **Qué hace:** imprime la lista `x` con formato f-string.
* **Propósito:** visualizar los tokens de entrada seleccionados.

**Línea 5**

```python
print(f'y: {y}')
```

* **Qué hace:** imprime la lista `y`, también con formato f-string.
* **Propósito:** comprobar la correspondencia entre cada token de entrada en `x` y su token siguiente en `y`.
* **Resultado típico:**

  ```
  x: [19204, 345, 82, 290]
  y: [345, 82, 290, 50256]
  ```

  donde `y` está desplazada una posición con respecto a `x`.


In [ ]:
for i in range(1, context_size + 1):

  context = enc_sample[:i]
  desired = enc_sample[i]

  print(context, "---->", desired)

**Línea 1**

```python
for i in range(1, context_size + 1):
```

* **Qué hace:** inicia un bucle `for` que itera sobre los valores enteros desde `1` hasta `context_size` inclusive.
* **Firma de la función:** `range(start, stop)` genera una secuencia de enteros comenzando en `start` y terminando en `stop - 1`.
* **Parámetros:**

  * `start = 1`: valor inicial de la secuencia.
  * `stop = context_size + 1`: asegura que el último valor del bucle sea igual a `context_size`.
* **Comportamiento:** el bucle recorre valores `1, 2, 3, 4` cuando `context_size = 4`.
* **Uso:** permite generar dinámicamente contextos de longitud creciente.

**Línea 2**

```python
context = enc_sample[:i]
```

* **Qué hace:** crea una sublista desde el inicio de `enc_sample` hasta el índice `i - 1`.
* **Comportamiento:** en cada iteración, `context` contiene los primeros `i` tokens.

  * En la primera iteración, 1 token.
  * En la segunda, 2 tokens.
  * Y así sucesivamente.
* **Propósito:** simular cómo el modelo ve un contexto progresivamente mayor de tokens antes de predecir el siguiente.

**Línea 3**

```python
desired = enc_sample[i]
```

* **Qué hace:** selecciona el token siguiente al último del contexto actual.
* **Explicación:**

  * Si `context = enc_sample[:i]`, el siguiente token está en la posición `i`.
* **Resultado:** `desired` contiene el token que el modelo debería predecir dada la secuencia `context`.

**Línea 4**

```python
print(context, "---->", desired)
```

* **Qué hace:** imprime el par formado por la lista `context` y su token objetivo `desired`.
* **Salida típica:**

  ```
  [123] ----> 456
  [123, 456] ----> 789
  [123, 456, 789] ----> 321
  [123, 456, 789, 321] ----> 654
  ```
* **Propósito:** visualizar de forma clara la relación entre cada contexto y el siguiente token que el modelo debe aprender a predecir.


In [ ]:
for i in range(1, context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

**Línea 1**

```python
for i in range(1, context_size+1):
```

* **Qué hace:** inicia un bucle que recorre los valores enteros desde `1` hasta `context_size` inclusive.
* **Parámetros de `range()`:**

  * `start = 1`: valor inicial.
  * `stop = context_size + 1`: el límite superior no se incluye, por eso se suma 1 para abarcar el valor máximo.
* **Comportamiento:** si `context_size = 4`, los valores de `i` serán `1, 2, 3, 4`.
* **Propósito:** generar contextos de longitud progresiva para ilustrar cómo el modelo aprende dependencias token a token.

**Línea 2**

```python
context = enc_sample[:i]
```

* **Qué hace:** selecciona una porción de la lista `enc_sample` desde el índice `0` hasta `i - 1`.
* **Resultado:** `context` contiene los primeros `i` tokens de la secuencia.
* **Propósito:** representar el texto que el modelo “ha visto” hasta ese punto.

**Línea 3**

```python
desired = enc_sample[i]
```

* **Qué hace:** toma el token que sigue inmediatamente después del contexto actual.
* **Resultado:** `desired` es el token objetivo (el siguiente token a predecir).
* **Importancia:** cada par `(context, desired)` corresponde a un ejemplo de entrenamiento del modelo de lenguaje.

**Línea 4**

```python
print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))
```

* **Qué hace:** decodifica ambos, el contexto y el token objetivo, para mostrarlos como texto legible.
* **Parámetros del método `decode()`:**

  * `tokens: list[int]`: lista de IDs a convertir a texto.
  * `disallowed_special` (opcional, por defecto `set()`): define si se omiten *special tokens*.
* **Comportamiento:**

  * `tokenizer.decode(context)` reconstruye el texto correspondiente a los tokens del contexto.
  * `tokenizer.decode([desired])` convierte el siguiente token en su representación textual.
* **Salida típica:**

  ```
  A ----> k
  Ak ----> w
  Akw ----> i
  Akwi ----> r
  ```
* **Propósito:** visualizar de forma comprensible cómo crece el contexto y cuál es el siguiente carácter o palabra que el modelo debe predecir en cada paso.


# **Listing 2.5 A dataset for batched inputs and targets**
# **Listado 2.5 Un conjunto de datos para entradas y objetivos agrupados**

In [16]:
# Dataset Management

import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)  #A Tokenizar todo el texto

    for i in range(0, len(token_ids) - max_length, stride):  #B Usar una ventana deslizante para dividir el libro en secuencias superpuestas de longitud máxima
      input_chunk = token_ids[i:i + max_length]
      target_chunk = token_ids[i + 1: i + max_length + 1]

      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):  #C Devolver el número total de filas del conjunto de datos
    return len(self.input_ids)

  def __getitem__(self, idx):  #D Devolver una sola fila del conjunto de datos
    return self.input_ids[idx], self.target_ids[idx]


**Línea 1**

```python
import torch
from torch.utils.data import Dataset, DataLoader
```

* **Qué hace:** importa PyTorch y los componentes de manejo de datos.
* `Dataset`: clase base para crear conjuntos de datos personalizados.
* `DataLoader`: utilidad para cargar datos por lotes, barajarlos y alimentarlos al modelo durante el entrenamiento.
* **Uso:** permite definir la estructura de los datos y cómo serán iterados en el entrenamiento de un LLM.

---

**Línea 2**

```python
class GPTDatasetV1(Dataset):
```

* **Qué hace:** define una clase llamada `GPTDatasetV1` que hereda de `torch.utils.data.Dataset`.
* **Propósito:** crear un conjunto de datos específico para entrenamiento de un modelo tipo GPT.
* **Requisito:** al heredar de `Dataset`, debe implementar obligatoriamente los métodos `__len__()` y `__getitem__()`.

---

**Línea 3**

```python
def __init__(self, txt, tokenizer, max_length, stride):
```

* **Qué hace:** constructor de la clase. Inicializa los atributos del dataset.
* **Parámetros:**

  * `txt`: texto completo a tokenizar y segmentar.
  * `tokenizer`: objeto `tiktoken.Encoding` o similar para convertir texto en IDs de tokens.
  * `max_length`: tamaño máximo de cada secuencia de entrada (*context length*).
  * `stride`: paso con el que se desliza la ventana de segmentación.
* **Ejemplo:** si `max_length=8` y `stride=4`, las ventanas se superpondrán parcialmente.

---

**Línea 4**

```python
self.input_ids = []
self.target_ids = []
```

* **Qué hace:** inicializa dos listas vacías para almacenar los tensores de entrada (`input_ids`) y los tensores objetivo (`target_ids`).
* **Uso:** cada par (`input_ids[i]`, `target_ids[i]`) representará una muestra de entrenamiento.

---

**Línea 5**

```python
token_ids = tokenizer.encode(txt)
```

* **Qué hace:** convierte el texto completo en una lista de IDs de tokens.
* **Salida:** `token_ids` es una lista de enteros que codifica el texto según el vocabulario del tokenizador.
* **Uso:** base para generar fragmentos de longitud `max_length`.

---

**Línea 6**

```python
for i in range(0, len(token_ids) - max_length, stride):
```

* **Qué hace:** recorre el texto tokenizado en pasos de tamaño `stride`.
* **Parámetros del bucle:**

  * `start = 0`: inicio del texto.
  * `stop = len(token_ids) - max_length`: evita exceder el final de la lista.
  * `step = stride`: controla la superposición entre fragmentos.
* **Comportamiento:** genera ventanas de tokens deslizantes del tamaño `max_length`.

---

**Línea 7**

```python
input_chunk = token_ids[i:i + max_length]
target_chunk = token_ids[i + 1: i + max_length + 1]
```

* **Qué hace:** crea dos listas de tokens:

  * `input_chunk`: secuencia de entrada.
  * `target_chunk`: secuencia desplazada un token hacia adelante.
* **Propósito:** entrenar el modelo para predecir el siguiente token (aprendizaje autoregresivo).
* **Relación:** `target_chunk[t]` es el token que el modelo debe predecir tras ver `input_chunk[:t+1]`.

---

**Línea 8**

```python
self.input_ids.append(torch.tensor(input_chunk))
self.target_ids.append(torch.tensor(target_chunk))
```

* **Qué hace:** convierte cada fragmento de IDs en tensores de PyTorch y los agrega a las listas correspondientes.
* **Tipo:** `torch.tensor(list[int]) → Tensor(dtype=torch.int64)`
* **Uso:** facilita el entrenamiento posterior en GPU, ya que los tensores son el formato estándar en PyTorch.

---

**Línea 9**

```python
def __len__(self):
    return len(self.input_ids)
```

* **Qué hace:** devuelve el número total de ejemplos en el dataset.
* **Requisito:** método obligatorio para que PyTorch pueda conocer el tamaño del conjunto de datos.
* **Salida:** número de pares `(input, target)` almacenados.

---

**Línea 10**

```python
def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]
```

* **Qué hace:** permite acceder a un ejemplo específico del dataset por índice.
* **Parámetro:**

  * `idx`: posición entera del ejemplo solicitado.
* **Salida:** tupla `(input_tensor, target_tensor)`.
* **Uso:** es el método que PyTorch llama internamente cuando el `DataLoader` itera sobre el dataset.

---

**Resumen técnico:**
Esta clase transforma texto continuo en pares de tensores de entrenamiento listos para un modelo autoregresivo (GPT).
Cada muestra contiene:

* **Entrada (`input_ids`)**: una secuencia de `max_length` tokens.
* **Objetivo (`target_ids`)**: la misma secuencia desplazada un token hacia adelante.

El parámetro `stride` controla el solapamiento entre fragmentos y, por tanto, la cantidad de ejemplos generados.


# **Listing 2.6 A data loader to generate batches with input-with pairs**

In [17]:
# Dataloader management

def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, num_workers=0):
  tokenizer = tiktoken.get_encoding('gpt2')  #A inicializamos el tokenizador
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)  #B creamos el dataset

  dataloader = DataLoader(
      dataset,
      batch_size = batch_size,
      shuffle = shuffle,
      drop_last = True,  #C drp_last = True Se descarta el último lote si es más corto que el tamaño de lote especificado para evitar picos de pérdida durante el entrenamiento.
      num_workers = 0  #D la cantidad de procesos de CPU que se utilizarán para el prprocesamiento
    )

  return dataloader

**Línea 1**

```python
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, num_workers=0):
```

* **Qué hace:** define una función llamada `create_dataloader_v1` que construye y devuelve un objeto `DataLoader` de PyTorch configurado para procesar texto.
* **Parámetros:**

  * `txt`: texto fuente que se tokenizará y segmentará.
  * `batch_size` (opcional, por defecto `4`): número de ejemplos por lote durante el entrenamiento.
  * `max_length` (opcional, por defecto `256`): número máximo de tokens por secuencia.
  * `stride` (opcional, por defecto `128`): desplazamiento entre ventanas sucesivas de texto tokenizado.
  * `shuffle` (opcional, por defecto `True`): si `True`, mezcla las muestras en cada época.
  * `num_workers` (opcional, por defecto `0`): número de subprocesos para cargar datos en paralelo (en Google Colab o CPU limitada suele mantenerse en `0`).
* **Propósito:** encapsular en una sola función la creación del dataset y del `DataLoader` configurado.

---

**Línea 2**

```python
tokenizer = tiktoken.get_encoding('gpt2')
```

* **Qué hace:** obtiene el objeto `Encoding` correspondiente al tokenizador GPT-2 de la librería `tiktoken`.
* **Salida:** instancia lista para convertir texto a IDs y viceversa.
* **Motivo:** garantizar que el dataset use el mismo esquema de codificación que el modelo a entrenar.

---

**Línea 3**

```python
dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
```

* **Qué hace:** crea una instancia de la clase `GPTDatasetV1` definida anteriormente.
* **Parámetros:**

  * `txt`: texto completo.
  * `tokenizer`: tokenizador cargado.
  * `max_length`: tamaño del contexto por muestra.
  * `stride`: desplazamiento entre secuencias.
* **Salida:** objeto `dataset` que implementa `__len__` y `__getitem__`.
* **Uso:** contiene todas las muestras (pares de entrada y objetivo) derivadas del texto.

---

**Línea 4**

```python
dataloader = DataLoader(
    dataset,
    batch_size = batch_size,
    shuffle = shuffle,
    drop_last = True,
    num_workers = 0
)
```

* **Qué hace:** crea un `DataLoader`, componente de PyTorch que organiza la entrega de datos al modelo en mini-lotes.
* **Parámetros:**

  * `dataset`: el objeto `GPTDatasetV1` con los ejemplos preparados.
  * `batch_size`: número de ejemplos por lote.
  * `shuffle`: mezcla las muestras al comienzo de cada época si está en `True`.
  * `drop_last=True`: descarta el último lote si tiene menos elementos que `batch_size`; útil para mantener dimensiones constantes.
  * `num_workers`: número de procesos de carga paralela de datos (por simplicidad, aquí se usa `0`).
* **Comportamiento:** el `DataLoader` usa internamente el método `__getitem__` del dataset para devolver pares `(input_ids, target_ids)` empaquetados por lotes.

---

**Línea 5**

```python
return dataloader
```

* **Qué hace:** devuelve el objeto `DataLoader` completamente configurado.
* **Uso:** este `dataloader` se puede iterar directamente en un bucle de entrenamiento con:

  ```python
  for x, y in dataloader:
      ...
  ```
* **Propósito:** simplificar la creación del flujo de datos de entrenamiento a partir de texto crudo.


In [18]:
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=1,
    max_length=4,
    stride=1,
    shuffle=False
)

data_iter = iter(dataloader)  #A convierte un cargador de datos en un iterador de Python para obtener la siguiente entrada a través de la función next() incorporada de Python
first_batch = next(data_iter)
print(first_batch)


NameError: name 'tiktoken' is not defined

**Línea 1**

```python
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=1,
    max_length=4,
    stride=1,
    shuffle=False
)
```

* **Qué hace:** crea un `DataLoader` usando la función `create_dataloader_v1`.
* **Parámetros enviados:**

  * `raw_text`: texto original cargado del archivo.
  * `batch_size=1`: cada lote contendrá solo una muestra `(input_ids, target_ids)`.
  * `max_length=4`: cada secuencia tendrá 4 tokens de longitud.
  * `stride=1`: la ventana deslizante avanza de un token a la vez, generando ejemplos solapados.
  * `shuffle=False`: mantiene el orden secuencial de las muestras (sin barajar).
* **Resultado:** se crea un `DataLoader` que contiene todas las secuencias generadas a partir del texto tokenizado, preparadas para ser recorridas en entrenamiento o inspección.

---

**Línea 2**

```python
data_iter = iter(dataloader)
```

* **Qué hace:** convierte el `DataLoader` en un iterador de Python.
* **Firma interna:** `iter(obj)` llama al método `__iter__()` del objeto si está definido.
* **Comportamiento:**

  * Permite recorrer el `DataLoader` manualmente con la función `next()`.
  * Cada llamada a `next(data_iter)` devolverá el siguiente lote de datos (`x, y`).
* **Uso:** útil para inspeccionar manualmente los primeros lotes o depurar el flujo de datos.

---

**Línea 3**

```python
first_batch = next(data_iter)
```

* **Qué hace:** obtiene el primer lote del iterador `data_iter`.
* **Mecanismo:** la función incorporada `next()` ejecuta la primera iteración del `DataLoader`.
* **Salida:** una tupla `(input_ids, target_ids)` donde:

  * `input_ids`: tensor de forma `[batch_size, max_length]` → aquí `[1, 4]`.
  * `target_ids`: tensor de forma idéntica, desplazado un token.
* **Uso:** acceder explícitamente al primer lote sin recorrer todo el dataloader.

---

**Línea 4**

```python
print(first_batch)
```

* **Qué hace:** imprime el contenido del primer lote obtenido.
* **Salida típica:**

  ```
  (tensor([[  40,  345,  290,  123]]), tensor([[ 345,  290,  123,  456]]))
  ```
* **Interpretación:**

  * El primer tensor representa la secuencia de entrada.
  * El segundo tensor es la misma secuencia desplazada un token a la derecha, objetivo del modelo.
* **Propósito:** verificar que el `DataLoader` está generando correctamente los pares de entrenamiento `(x, y)` según los parámetros especificados.


In [ ]:
second_batch = next(data_iter)
print(second_batch)

**Línea 1**

```python
second_batch = next(data_iter)
```

* **Qué hace:** obtiene el siguiente lote del iterador `data_iter` creado a partir del `DataLoader`.
* **Mecanismo:** la función `next()` llama internamente al método `__next__()` del iterador, devolviendo el siguiente elemento de la secuencia.
* **Comportamiento:**

  * En la primera llamada (`first_batch = next(data_iter)`), se extrajo el primer lote.
  * En esta segunda llamada, se extrae el siguiente lote disponible.
* **Salida:** una tupla `(input_ids, target_ids)` donde cada tensor tiene tamaño `[batch_size, max_length]`.
* **Ejemplo de salida:**

  ```
  (tensor([[ 345,  290,  123,  456]]), tensor([[ 290,  123,  456,  789]]))
  ```
* **Interpretación:** el nuevo lote representa la siguiente ventana deslizante de tokens según el `stride=1` configurado, es decir, el contexto se desplazó un token respecto al lote anterior.

**Línea 2**

```python
print(second_batch)
```

* **Qué hace:** imprime el contenido del segundo lote.
* **Propósito:** inspeccionar el comportamiento del `DataLoader` y confirmar que genera las secuencias de entrenamiento consecutivas correctamente.
* **Resultado:** muestra los tensores de entrada y sus correspondientes objetivos, lo que permite verificar que el dataset mantiene coherencia y secuencia en los datos.


In [ ]:
third_batch = next(data_iter)
print(third_batch)

**Línea 1**

```python
third_batch = next(data_iter)
```

* **Qué hace:** solicita el tercer lote del iterador `data_iter` asociado al `DataLoader`.
* **Comportamiento:**

  * Cada llamada a `next(data_iter)` avanza una posición en el flujo de lotes.
  * Dado que el `stride` se configuró en `1`, este lote estará desplazado un token más respecto al segundo lote.
* **Salida:** tupla `(input_ids, target_ids)` donde ambos son tensores de forma `[batch_size, max_length]`.
* **Ejemplo de salida:**

  ```
  (tensor([[290, 123, 456, 789]]), tensor([[123, 456, 789, 321]]))
  ```
* **Interpretación:**

  * El primer tensor contiene los tokens de entrada del tercer fragmento del texto.
  * El segundo tensor contiene los tokens objetivo correspondientes, desplazados un paso hacia adelante.
* **Propósito:** comprobar que el `DataLoader` genera correctamente la secuencia continua de ejemplos autoregresivos, avanzando un token en cada lote cuando `stride=1`.

**Línea 2**

```python
print(third_batch)
```

* **Qué hace:** imprime el contenido del tercer lote.
* **Uso:** inspección manual de los datos generados, confirmando que los pares `(x, y)` siguen la lógica de “contexto → siguiente token” establecida en el diseño del dataset.


# EXERCISE 2.2 DATA LOADERS WITH DIFFERENT STRIDES AND CONTEXT SIZES:
To develop more intuition for how the data loader works, try to run it with different
settings such as:
*   max_length=2 and stride=2
*   max_length=8 and stride=2.

---

# EJERCICIO 2.2 CARGADORES DE DATOS CON DIFERENTES PASOS Y TAMAÑOS DE CONTEXTO:
Para comprender mejor el funcionamiento del cargador de datos, intente ejecutarlo con diferentes configuraciones, como:
*   max_length=2 y stride=2
*   max_length=8 y stride=2

In [ ]:
dataloader_bis = create_dataloader_v1(
    raw_text,
    batch_size=1,
    max_length=2,
    stride=2,
    shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

**Línea 1**

```python
dataloader_bis = create_dataloader_v1(
    raw_text,
    batch_size=1,
    max_length=2,
    stride=2,
    shuffle=False
)
```

* **Qué hace:** crea un nuevo `DataLoader` con parámetros distintos al anterior.
* **Parámetros enviados:**

  * `raw_text`: texto original del archivo.
  * `batch_size=1`: genera un lote con una sola muestra `(input_ids, target_ids)`.
  * `max_length=2`: cada secuencia de entrada contendrá 2 tokens.
  * `stride=2`: la ventana deslizante avanza de dos en dos, sin superposición entre fragmentos.
  * `shuffle=False`: mantiene el orden secuencial de las muestras.
* **Resultado:** `dataloader_bis` es un cargador de datos que recorre el texto en segmentos consecutivos no solapados de longitud 2.
  Ejemplo conceptual:

  ```
  tokens = [1, 2, 3, 4, 5, 6]
  con stride=2 → [(1,2)->(2,3)], [(3,4)->(4,5)], [(5,6)->(6,7)]
  ```

---

**Línea 2**

```python
data_iter = iter(dataloader)
```

* **Qué hace:** crea un iterador a partir del `DataLoader` llamado `dataloader`.
* **Observación importante:** aquí se usa **`dataloader`**, no `dataloader_bis`.
  Esto significa que el iterador sigue apuntando al cargador anterior (el que tenía `max_length=4` y `stride=1`), no al nuevo.
* **Comportamiento:** se reutiliza el cargador previo, por lo tanto los datos obtenidos no corresponden a la nueva configuración.

---

**Línea 3**

```python
first_batch = next(data_iter)
```

* **Qué hace:** obtiene el siguiente lote disponible del iterador `data_iter`.
* **Resultado:** devuelve una tupla `(input_ids, target_ids)` del `dataloader` anterior, no de `dataloader_bis`, debido a la referencia usada en la línea anterior.

---

**Línea 4**

```python
print(first_batch)
```

* **Qué hace:** imprime el lote obtenido.
* **Contenido:** dos tensores (`input_ids`, `target_ids`) de tamaño `[1, 4]`, porque provienen del `DataLoader` inicial con `max_length=4`.
* **Corrección recomendada:** si se pretende visualizar el primer lote del nuevo `dataloader_bis`, debe escribirse:

  ```python
  data_iter = iter(dataloader_bis)
  first_batch = next(data_iter)
  print(first_batch)
  ```
* **Propósito:** inspeccionar el comportamiento del nuevo cargador de datos, que debería generar secuencias sin solapamiento (por el `stride=2`).


In [ ]:
second_batch = next(data_iter)
print(second_batch)

In [ ]:
dataloader_bis = create_dataloader_v1(
    raw_text,
    batch_size=1,
    max_length=8,
    stride=2,
    shuffle=False
)

data_iter = iter(dataloader_bis)
first_batch = next(data_iter)
print(first_batch)

**Línea 1**

```python
dataloader_bis = create_dataloader_v1(
    raw_text,
    batch_size=1,
    max_length=8,
    stride=2,
    shuffle=False
)
```

* **Qué hace:** crea un nuevo `DataLoader` con configuración diferente para generar fragmentos más largos de texto tokenizado.
* **Parámetros:**

  * `raw_text`: texto completo que será tokenizado.
  * `batch_size=1`: cada lote contiene una sola muestra `(input_ids, target_ids)`.
  * `max_length=8`: cada secuencia de entrada contiene 8 tokens consecutivos.
  * `stride=2`: la ventana deslizante avanza 2 tokens por iteración, por lo que hay superposición de 6 tokens entre fragmentos consecutivos.
  * `shuffle=False`: mantiene el orden original del texto, sin mezclar las muestras.
* **Resultado:** `dataloader_bis` recorre el texto en fragmentos parcialmente solapados de longitud 8, adecuados para entrenar modelos autoregresivos.

---

**Línea 2**

```python
data_iter = iter(dataloader_bis)
```

* **Qué hace:** convierte el `DataLoader` en un iterador de Python.
* **Comportamiento:**

  * Permite usar `next(data_iter)` para extraer manualmente el siguiente lote.
  * Cada lote contiene un par `(input_ids, target_ids)` de tensores con dimensiones `[batch_size, max_length]`.
* **Uso:** facilita inspeccionar de forma controlada las primeras muestras generadas por el cargador.

---

**Línea 3**

```python
first_batch = next(data_iter)
```

* **Qué hace:** obtiene el primer lote del iterador `data_iter`.
* **Salida:** tupla `(input_ids, target_ids)` donde:

  * `input_ids`: tensor con los primeros 8 tokens de la secuencia de entrada.
  * `target_ids`: tensor con los 8 tokens siguientes, desplazados una posición a la derecha.
* **Forma de salida:** `[1, 8]` en ambos tensores, ya que `batch_size=1` y `max_length=8`.
* **Propósito:** verificar la correcta generación del primer par de datos para entrenamiento.

---

**Línea 4**

```python
print(first_batch)
```

* **Qué hace:** muestra en pantalla el contenido del lote obtenido.
* **Salida típica:**

  ```
  (tensor([[  40,  345,  290,  123,  567,  901,  456,  789]]),
   tensor([[ 345,  290,  123,  567,  901,  456,  789,  654]]))
  ```
* **Interpretación:**

  * El primer tensor (`input_ids`) es el contexto que el modelo ve.
  * El segundo tensor (`target_ids`) son los tokens que el modelo debe predecir.
* **Conclusión:** confirma que el `DataLoader` genera correctamente secuencias de longitud 8 con desplazamiento de 1 token y superposición controlada por `stride=2`.


In [ ]:
second_batch = next(data_iter)
print(second_batch)

# **...how we can use the data loader to sample with a batch size greater than 1:**

# **...cómo podemos utilizar el cargador de datos para muestrear con un tamaño de lote mayor a 1:**

In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f'Inputs: {inputs}')
print(f'Target: {targets}')

**Línea 1**

```python
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)
```

* **Qué hace:** crea un `DataLoader` configurado para procesar el texto `raw_text` en lotes de tamaño 8.
* **Parámetros:**

  * `raw_text`: texto de entrada completo.
  * `batch_size=8`: cada lote contendrá 8 pares `(input_ids, target_ids)`.
  * `max_length=4`: cada secuencia de entrada tendrá 4 tokens.
  * `stride=4`: el desplazamiento entre ventanas es igual al tamaño del contexto, por lo que **no hay superposición** entre fragmentos consecutivos.
* **Resultado:** el `DataLoader` recorre el texto dividiéndolo en fragmentos contiguos de 4 tokens, ideales para entrenamientos o pruebas rápidas.

---

**Línea 2**

```python
data_iter = iter(dataloader)
```

* **Qué hace:** crea un iterador Python a partir del `DataLoader`.
* **Comportamiento:**

  * Permite recorrer el `DataLoader` con `next()` o en un bucle `for`.
  * Cada iteración devuelve un lote de datos con la forma `(inputs, targets)`.
* **Uso:** acceder manualmente a los primeros lotes para inspeccionar su contenido.

---

**Línea 3**

```python
inputs, targets = next(data_iter)
```

* **Qué hace:** obtiene el primer lote de datos del iterador.
* **Salida:** dos tensores (`inputs`, `targets`) con dimensiones `[batch_size, max_length]`, es decir, `[8, 4]`.
* **Contenido:**

  * `inputs`: secuencias de 4 tokens consecutivos.
  * `targets`: las mismas secuencias desplazadas un token hacia adelante (tokens objetivo).
* **Propósito:** verificar cómo el `DataLoader` genera y agrupa las secuencias de texto tokenizadas.

---

**Línea 4**

```python
print(f'Inputs: {inputs}')
```

* **Qué hace:** imprime el tensor que contiene las secuencias de entrada del primer lote.
* **Ejemplo de salida:**

  ```
  Inputs: tensor([[  40,  345,  290,  123],
                  [ 567,  901,  456,  789],
                  [ ... ],
                  ...])
  ```
* **Interpretación:** cada fila del tensor representa una secuencia independiente de 4 tokens.

---

**Línea 5**

```python
print(f'Target: {targets}')
```

* **Qué hace:** imprime el tensor que contiene los tokens objetivo correspondientes a cada secuencia de entrada.
* **Ejemplo de salida:**

  ```
  Target: tensor([[ 345,  290,  123,  567],
                  [ 901,  456,  789,  654],
                  [ ... ],
                  ...])
  ```
* **Relación:** cada fila de `targets` está alineada con la de `inputs`, desplazada un token hacia la derecha.
* **Propósito:** comprobar visualmente que el par `(input, target)` cumple la estructura autoregresiva usada en el entrenamiento de modelos tipo GPT.


# **2.7 Creating token embeddings**
# **2.7 Creación de incrustaciones de tokens**

In [ ]:
import torch
w = torch.tensor(3.0, requires_grad=True)
x = torch.tensor(2.0)
y = w * x        # forward
loss = (y - 10)**2
loss.backward()  # backward

print(w.grad)    # d(loss)/d(w)

**Línea 1**

```python
import torch
```

* **Qué hace:** importa el módulo principal de PyTorch.
* **Propósito:** habilitar el uso de tensores y operaciones con seguimiento automático de gradientes mediante *autograd*.

---

**Línea 2**

```python
w = torch.tensor(3.0, requires_grad=True)
```

* **Qué hace:** crea un tensor escalar con valor `3.0`.
* **Parámetro clave:**

  * `requires_grad=True`: indica que PyTorch debe rastrear todas las operaciones realizadas sobre `w` para calcular derivadas automáticas.
* **Resultado:** `w` es una variable diferenciable, por lo que su gradiente (`w.grad`) se actualizará cuando se llame a `backward()`.

---

**Línea 3**

```python
x = torch.tensor(2.0)
```

* **Qué hace:** crea un tensor escalar con valor `2.0`.
* **Parámetro `requires_grad`:** no se especifica, por lo tanto su valor por defecto es `False`.
* **Resultado:** `x` actúa como una constante en las operaciones posteriores.

---

**Línea 4**

```python
y = w * x
```

* **Qué hace:** multiplica los tensores `w` y `x` (operación *forward*).
* **Resultado:** `y = 3.0 × 2.0 = 6.0`.
* **Importante:** la operación se registra en el grafo computacional de PyTorch porque `w` tiene `requires_grad=True`.
* **Forma simbólica:** `y = f(w) = w * 2`.

---

**Línea 5**

```python
loss = (y - 10)**2
```

* **Qué hace:** calcula una función de pérdida cuadrática.
* **Expresión matemática:**
  [
  \text{loss} = (w \times x - 10)^2
  ]
* **Resultado numérico:**
  [
  (6 - 10)^2 = 16
  ]
* **El grafo de cómputo** ahora incluye todas las operaciones necesarias para obtener el gradiente de `loss` respecto a `w`.

---

**Línea 6**

```python
loss.backward()
```

* **Qué hace:** realiza la propagación hacia atrás (*backpropagation*).
* **Efecto:** PyTorch calcula
  [
  \frac{d(\text{loss})}{dw}
  ]
  y lo almacena en `w.grad`.
* **Cálculo manual:**
  [
  \text{loss} = (w \times 2 - 10)^2
  ]
  Derivando respecto a `w`:
  [
  \frac{d(\text{loss})}{dw} = 2(w \times 2 - 10) \times 2 = 4(2w - 10)
  ]
  Sustituyendo `w = 3`:
  [
  4(6 - 10) = 4(-4) = -16
  ]
* **Resultado esperado:** `w.grad = tensor(-16.)`.

---

**Línea 7**

```python
print(w.grad)
```

* **Qué hace:** imprime el gradiente almacenado en `w.grad`.
* **Salida:**

  ```
  tensor(-16.)
  ```
* **Interpretación:** el gradiente negativo indica que, para minimizar la pérdida, el valor de `w` debería **aumentar**, lo cual concuerda con la dirección del descenso del gradiente.


In [ ]:
input_ids = torch.tensor([2, 3, 5, 1])

vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

print(embedding_layer.weight)

**Línea 1**

```python
input_ids = torch.tensor([2, 3, 5, 1])
```

* **Qué hace:** crea un tensor con los índices `[2, 3, 5, 1]`.
* **Interpretación:** cada número representa el **ID de un token** dentro del vocabulario.
* **Uso:** estos IDs se usarán para consultar las representaciones vectoriales (embeddings) correspondientes en la capa de incrustación.
* **Tipo de dato:** por defecto, `torch.int64`, que es el formato requerido por `nn.Embedding`.

---

**Línea 2**

```python
vocab_size = 6
```

* **Qué hace:** define el tamaño del vocabulario, es decir, cuántos tokens distintos hay.
* **Valor:** `6` implica que los tokens válidos van de `0` a `5`.
* **Importancia:** la capa `Embedding` solo puede indexar IDs dentro de ese rango.

---

**Línea 3**

```python
output_dim = 3
```

* **Qué hace:** define la dimensión de los vectores de embedding.
* **Significado:** cada token se representará mediante un vector de 3 valores (dimensión de espacio latente).
* **Ejemplo:** si `output_dim=3`, el token con ID `2` se mapeará a algo como `[0.4, -0.1, 0.9]`.

---

**Línea 4**

```python
torch.manual_seed(123)
```

* **Qué hace:** fija la **semilla aleatoria** de PyTorch.
* **Propósito:** asegurar reproducibilidad; cada inicialización aleatoria será igual en futuras ejecuciones.
* **Uso:** importante para depuración o comparación de resultados.

---

**Línea 5**

```python
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
```

* **Qué hace:** crea una **capa de embedding**.
* **Firma:**
  `torch.nn.Embedding(num_embeddings, embedding_dim)`
* **Parámetros:**

  * `num_embeddings = vocab_size`: cantidad de tokens únicos (aquí 6).
  * `embedding_dim = output_dim`: tamaño de cada vector de embedding (aquí 3).
* **Funcionamiento:**

  * Internamente contiene una matriz de pesos de forma `[vocab_size, output_dim]`.
  * Cada fila corresponde al vector de embedding de un token.
* **Inicialización:** los valores se generan aleatoriamente según una distribución uniforme en el rango `(-1/sqrt(embedding_dim), 1/sqrt(embedding_dim))`.
* **Uso posterior:** al pasar `input_ids` a esta capa, se seleccionarán las filas correspondientes.

---

**Línea 6**

```python
print(embedding_layer.weight)
```

* **Qué hace:** imprime la matriz de pesos de la capa de embedding.
* **Tamaño:** `[6, 3]` (una fila por token, tres valores por embedding).
* **Ejemplo de salida reproducible:**

  ```
  Parameter containing:
  tensor([[ 0.3374,  0.2079,  0.1385],
          [ 0.2542, -0.3456, -0.1756],
          [-0.2624, -0.2777, -0.2861],
          [ 0.2369, -0.2038,  0.3054],
          [ 0.1227,  0.3397,  0.0446],
          [ 0.1266, -0.2451, -0.2848]], requires_grad=True)
  ```
* **Interpretación:** cada fila representa la **representación vectorial aprendible** de un token.
* **Importancia:** estos vectores son parámetros entrenables (`requires_grad=True`), y se actualizarán durante el entrenamiento para capturar relaciones semánticas entre tokens.


In [ ]:
print(embedding_layer(torch.tensor([3])))

**Línea 1**

```python
print(embedding_layer(torch.tensor([3])))
```

* **Qué hace:** pasa el tensor `[3]` como entrada a la capa de embedding `embedding_layer`.

* **Entrada:**

  * `torch.tensor([3])` es un tensor con un único índice de token (`ID = 3`).
  * El valor `3` corresponde a la **cuarta fila** de la matriz de pesos del embedding (los índices comienzan en 0).
  * Tipo de dato: debe ser entero (`torch.long` o `torch.int64`), ya que la capa realiza una búsqueda por índice.

* **Comportamiento interno:**

  * La capa `nn.Embedding` actúa como una **tabla de búsqueda** (*lookup table*).
  * Dado el índice `3`, retorna la fila número 3 de su matriz interna de pesos `embedding_layer.weight`.
  * No realiza operaciones matemáticas complejas; solo accede a la fila correspondiente.

* **Salida:**

  * Tensor de forma `[1, output_dim]` → `[1, 3]` en este caso.
  * Contiene el vector de embedding asociado al token con ID 3.
  * Ejemplo (con la semilla fijada en 123):

    ```
    tensor([[ 0.2369, -0.2038,  0.3054]], grad_fn=<EmbeddingBackward0>)
    ```

* **Propósito:**

  * Recuperar la representación vectorial del token ID 3.
  * Este vector se usará posteriormente como entrada para las capas posteriores del modelo (por ejemplo, capas de atención o MLP).


# Let's now apply that to all four input IDs we defined earlier
## (torch.tensor([2, 3, 5, 1])):

# Apliquemos ahora esto a los cuatro ID de entrada que definimos anteriormente:
## (torch.tensor([2, 3, 5, 1])):

In [ ]:
print(embedding_layer(input_ids))

**Línea 1**

```python
print(embedding_layer(input_ids))
```

* **Qué hace:** pasa todo el tensor `input_ids` (`[2, 3, 5, 1]`) a la capa de embedding `embedding_layer`.

* **Entrada:**

  * `input_ids` contiene los IDs de los tokens a representar.
  * Cada valor se usa como índice para seleccionar una fila en la matriz `embedding_layer.weight`, de tamaño `[vocab_size, output_dim] = [6, 3]`.
  * Los índices válidos van de `0` a `5`; todos los valores en `input_ids` están dentro de ese rango.

* **Comportamiento interno:**

  * La capa **realiza una búsqueda por índice** (lookup) para cada ID en `input_ids`.
  * Construye un tensor 2D donde cada fila es el vector de embedding correspondiente a un token.
  * Matemáticamente, no hay multiplicaciones ni sumas, solo acceso directo a las filas de la matriz de pesos.

* **Salida:**

  * Tensor de forma `[len(input_ids), output_dim]` → `[4, 3]`.
  * Cada fila es el vector de embedding asociado a los IDs `[2]`, `[3]`, `[5]`, `[1]` respectivamente.
  * Ejemplo con la semilla `123`:

    ```
    tensor([[-0.2624, -0.2777, -0.2861],
            [ 0.2369, -0.2038,  0.3054],
            [ 0.1266, -0.2451, -0.2848],
            [ 0.2542, -0.3456, -0.1756]], grad_fn=<EmbeddingBackward0>)
    ```

* **Interpretación:**

  * Cada fila es una **representación vectorial densa** (embedding) del token correspondiente.
  * Estas representaciones son **parámetros entrenables**, y durante el entrenamiento se ajustan para capturar relaciones semánticas entre tokens.
  * Este paso es fundamental en modelos de lenguaje, ya que convierte IDs discretos en vectores continuos que el modelo puede procesar.


In [ ]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

**Línea 1**

```python
vocab_size = 50257
```

* **Qué hace:** define el tamaño del vocabulario, es decir, cuántos tokens únicos existen en el modelo.
* **Contexto:** `50257` es el tamaño del vocabulario del modelo **GPT-2** (50 000 tokens base + algunos tokens especiales).
* **Uso:** este valor determina cuántas filas tendrá la matriz de embeddings, una por cada token posible.

---

**Línea 2**

```python
output_dim = 256
```

* **Qué hace:** establece la dimensión del espacio de embeddings.
* **Significado:** cada token se representará mediante un vector de **256 valores** en el espacio latente.
* **Contexto:** en modelos GPT reales, este número puede ser 768, 1024 o más, dependiendo del tamaño del modelo; aquí se usa 256 por simplicidad o para entrenamiento reducido.

---

**Línea 3**

```python
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
```

* **Qué hace:** crea una **capa de embedding** en PyTorch para representar tokens.

* **Firma:**

  ```python
  torch.nn.Embedding(num_embeddings, embedding_dim)
  ```

* **Parámetros:**

  * `num_embeddings = vocab_size` → número total de tokens posibles (50257).
  * `embedding_dim = output_dim` → tamaño de cada vector (256).

* **Inicialización:**

  * Crea una matriz de pesos de forma `[50257, 256]`.
  * Cada fila contiene el vector de embedding inicial de un token (valores iniciales aleatorios).
  * Los pesos son parámetros entrenables (`requires_grad=True`).

* **Propósito:**

  * Traducir **IDs discretos de tokens** en **vectores continuos** que el modelo pueda procesar.
  * Este embedding será la **primera capa** de un modelo tipo GPT, donde cada token del texto de entrada se convierte en un vector de 256 dimensiones antes de pasar a capas posteriores (atención, MLP, etc.).


In [ ]:
max_length = 4

dataloader = create_dataloader_v1(
    raw_text,
    batch_size=8,
    max_length=max_length,
    stride=max_length,
    shuffle=False
    )

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print(f'Token IDs: {inputs}')
print(f'Inputs shape: {inputs.shape}')


**Línea 1**

```python
max_length = 4
```

* **Qué hace:** define el tamaño del contexto o número máximo de tokens que tendrá cada secuencia de entrada.
* **Uso:** cada muestra del dataset contendrá exactamente 4 tokens consecutivos.

---

**Línea 2**

```python
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=8,
    max_length=max_length,
    stride=max_length,
    shuffle=False
)
```

* **Qué hace:** crea un `DataLoader` usando la función definida previamente.
* **Parámetros:**

  * `raw_text`: texto fuente completo que será tokenizado.
  * `batch_size=8`: número de muestras por lote.
  * `max_length=4`: longitud de cada secuencia de entrada.
  * `stride=max_length`: la ventana deslizante avanza 4 tokens en cada paso → **sin superposición** entre fragmentos.
  * `shuffle=False`: mantiene el orden original de las muestras.
* **Resultado:** el `DataLoader` recorre el texto dividiéndolo en secuencias contiguas de 4 tokens, agrupadas en lotes de 8 ejemplos.

---

**Línea 3**

```python
data_iter = iter(dataloader)
```

* **Qué hace:** convierte el `DataLoader` en un iterador Python.
* **Uso:** permite acceder manualmente al siguiente lote con `next(data_iter)`.

---

**Línea 4**

```python
inputs, targets = next(data_iter)
```

* **Qué hace:** obtiene el primer lote del iterador.
* **Salida:**

  * `inputs`: tensor con los IDs de tokens de las secuencias de entrada.
  * `targets`: tensor con los mismos IDs, desplazados un token hacia adelante (objetivos de predicción).
* **Dimensiones:** ambos tensores tienen forma `[batch_size, max_length]` → `[8, 4]`.

---

**Línea 5**

```python
print(f'Token IDs: {inputs}')
```

* **Qué hace:** imprime las secuencias de tokens que constituyen las entradas del primer lote.
* **Contenido:** cada fila contiene 4 IDs consecutivos de tokens del texto original.

---

**Línea 6**

```python
print(f'Inputs shape: {inputs.shape}')
```

* **Qué hace:** muestra la forma (dimensiones) del tensor `inputs`.
* **Salida esperada:**

  ```
  Inputs shape: torch.Size([8, 4])
  ```
* **Interpretación:** el lote contiene 8 secuencias (una por muestra), y cada secuencia tiene 4 tokens.
* **Propósito:** verificar que el `DataLoader` produce correctamente los lotes con el tamaño y estructura definidos.


In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

**Línea 1**

```python
token_embeddings = token_embedding_layer(inputs)
```

* **Qué hace:** pasa el tensor `inputs` (que contiene IDs de tokens) a través de la capa de embedding `token_embedding_layer`.
* **Entrada:**

  * `inputs`: tensor de tamaño `[batch_size, max_length]`, por ejemplo `[8, 4]`.
  * Cada valor en `inputs` es un índice entre `0` y `vocab_size - 1` (en este caso, `0–50256`).
* **Comportamiento interno:**

  * La capa `nn.Embedding` actúa como una **tabla de búsqueda (lookup table)**.
  * Por cada token ID en `inputs`, selecciona su vector correspondiente de la matriz de embeddings `token_embedding_layer.weight` (de tamaño `[vocab_size, output_dim] = [50257, 256]`).
  * Construye un nuevo tensor en el que cada token se reemplaza por su representación vectorial de 256 dimensiones.
* **Salida:**

  * Tensor `token_embeddings` de tamaño `[batch_size, max_length, output_dim]`.
  * En este caso: `[8, 4, 256]`.
  * Cada fila contiene 4 tokens (por muestra), y cada token está representado por un vector de 256 valores continuos.

---

**Línea 2**

```python
print(token_embeddings.shape)
```

* **Qué hace:** imprime las dimensiones del tensor `token_embeddings`.
* **Salida esperada:**

  ```
  torch.Size([8, 4, 256])
  ```
* **Interpretación:**

  * `8` → número de secuencias en el lote (*batch size*).
  * `4` → número de tokens por secuencia (*context length*).
  * `256` → dimensión del espacio de embeddings.
* **Propósito:** confirmar que la capa de embedding ha convertido correctamente los IDs enteros en vectores densos con la dimensionalidad especificada.


In [ ]:
context_length = max_length

pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

pos_embeddings = pos_embedding_layer(torch.arange(context_length))

print(pos_embeddings.shape)

**Línea 1**

```python
context_length = max_length
```

* **Qué hace:** asigna a `context_length` el mismo valor que `max_length` (en este caso, `4`).
* **Significado:** `context_length` representa el número máximo de posiciones que puede manejar el modelo dentro de una secuencia.
* **Uso:** servirá para crear los **embeddings posicionales**, que indican la posición de cada token en la secuencia.

---

**Línea 2**

```python
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
```

* **Qué hace:** crea una nueva capa de embedding para posiciones.
* **Firma:**

  ```python
  torch.nn.Embedding(num_embeddings, embedding_dim)
  ```
* **Parámetros:**

  * `num_embeddings = context_length`: número de posiciones posibles (aquí, 4).
  * `embedding_dim = output_dim`: dimensión del vector que representará cada posición (aquí, 256).
* **Resultado:** una matriz de tamaño `[4, 256]`, donde cada fila corresponde al vector de embedding de una posición (posición 0, 1, 2 y 3).
* **Propósito:** permitir que el modelo distinga tokens según su **orden** dentro de la secuencia, algo que los embeddings de tokens por sí solos no representan.

---

**Línea 3**

```python
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
```

* **Qué hace:** obtiene los embeddings para las posiciones `[0, 1, 2, 3]`.
* **Detalles:**

  * `torch.arange(context_length)` genera el tensor `[0, 1, 2, 3]`.
  * Cada índice se usa para recuperar su vector correspondiente de `pos_embedding_layer.weight`.
* **Salida:** tensor `pos_embeddings` de tamaño `[4, 256]`.
  Cada fila es un vector de 256 valores que representa una posición específica.

---

**Línea 4**

```python
print(pos_embeddings.shape)
```

* **Qué hace:** imprime las dimensiones del tensor de embeddings posicionales.
* **Salida esperada:**

  ```
  torch.Size([4, 256])
  ```
* **Interpretación:**

  * `4` → cantidad de posiciones en la secuencia (una por token).
  * `256` → dimensión del espacio de representación para cada posición.
* **Propósito:** confirmar que la capa de embeddings posicionales genera correctamente un vector de 256 valores para cada una de las 4 posiciones posibles.


expliacción

In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

**Línea 1**

```python
input_embeddings = token_embeddings + pos_embeddings
```

* **Qué hace:** suma los **embeddings de tokens** (`token_embeddings`) y los **embeddings posicionales** (`pos_embeddings`).

* **Entrada:**

  * `token_embeddings`: tensor de tamaño `[batch_size, context_length, output_dim]`, por ejemplo `[8, 4, 256]`.
  * `pos_embeddings`: tensor de tamaño `[context_length, output_dim]`, por ejemplo `[4, 256]`.

* **Comportamiento interno:**

  * PyTorch aplica *broadcasting*: el tensor `[4, 256]` se expande a `[8, 4, 256]` para poder realizar la suma elemento a elemento.
  * Así, a cada posición (columna temporal) de todas las secuencias del lote se le suma el mismo vector posicional.

* **Resultado:**

  * `input_embeddings` tiene forma `[8, 4, 256]`.
  * Cada token queda representado por la suma de su vector de significado (embedding de token) más el vector que indica su posición en la secuencia (embedding posicional).

* **Propósito:**

  * Incorporar información de **orden secuencial** al modelo.
  * Los embeddings de tokens por sí solos no contienen noción de posición; al sumarlos con los posicionales, el modelo puede distinguir el “primer” token del “último”.
  * Este tensor combinado se usa como entrada principal a las capas de atención del Transformer.

---

**Línea 2**

```python
print(input_embeddings.shape)
```

* **Qué hace:** imprime la forma del tensor resultante.
* **Salida esperada:**

  ```
  torch.Size([8, 4, 256])
  ```
* **Interpretación:**

  * `8` → tamaño del lote (*batch size*).
  * `4` → longitud del contexto (*context length*).
  * `256` → dimensión de cada vector de embedding.
* **Conclusión:** confirma que la suma se realizó correctamente y que cada token ahora tiene una representación enriquecida con información de posición.


***
***
***
# Aquí arranca el capítulo 3 del libro:
# 3 Codding Attention Mechanisms
# 3 Codificando el Mecanismo de Atención

In [20]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6
)

print(inputs)

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])


---

**Línea 1**

```python
import torch
```

* **Qué hace:** importa la biblioteca PyTorch, el marco principal que se utilizará para implementar y entrenar el modelo Transformer.
* **Propósito:** PyTorch proporciona:

  * Tensores (estructuras de datos similares a arrays de NumPy pero con soporte para GPU).
  * Operaciones matemáticas vectorizadas.
  * Mecanismos de autodiferenciación (`autograd`), fundamentales para el entrenamiento del modelo.
* **Contexto:** en esta sección del libro, PyTorch se usa para representar las matrices de embeddings, pesos de atención y salidas del modelo, simulando los cálculos internos de un bloque Transformer.

---

**Línea 2**

```python
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
     [0.55, 0.87, 0.66], # journey (x^2)
     [0.57, 0.85, 0.64], # starts (x^3)
     [0.22, 0.58, 0.33], # with (x^4)
     [0.77, 0.25, 0.10], # one (x^5)
     [0.05, 0.80, 0.55]] # step (x^6)
)
```

* **Qué hace:** define un tensor bidimensional que contiene las representaciones numéricas (embeddings) de seis palabras consecutivas.
* **Estructura:**

  * Cada **fila** corresponde a un token (una palabra o subpalabra).
  * Cada **columna** representa una dimensión del espacio de embeddings (en este caso, dimensión = 3).
* **Forma del tensor:** `[6, 3]`

  * `6` → número de tokens en la secuencia (“Your journey starts with one step”).
  * `3` → dimensión del vector que representa cada token (dimensión reducida para visualización; los LLM reales usan entre 256 y 12288).
* **Semántica:**

  * Estos vectores no provienen de un modelo real, sino que son valores arbitrarios para **ilustrar el flujo de datos** a través de las transformaciones de la capa de atención.
  * En un modelo real, estos valores serían la salida de la suma:
    [
    x_i = \text{TokenEmbedding}(w_i) + \text{PositionEmbedding}(i)
    ]
    donde `w_i` es el ID del token en la secuencia, y `i` su posición.
* **Propósito:** `inputs` servirá como entrada al mecanismo de **self-attention**, el componente central del Transformer.
  A partir de aquí, se derivarán las matrices **Q (queries)**, **K (keys)** y **V (values)** para cada token.
* **Tipo de dato:** flotante (`torch.float32`), adecuado para cálculos matriciales y gradientes.

---

**Línea 3**

```python
print(inputs)
```

* **Qué hace:** imprime el tensor `inputs` en consola.
* **Salida esperada:**

  ```
  tensor([[0.4300, 0.1500, 0.8900],
          [0.5500, 0.8700, 0.6600],
          [0.5700, 0.8500, 0.6400],
          [0.2200, 0.5800, 0.3300],
          [0.7700, 0.2500, 0.1000],
          [0.0500, 0.8000, 0.5500]])
  ```
* **Interpretación:**

  * Cada fila se puede considerar el vector `xᶦ`, correspondiente al token i-ésimo de la secuencia.
  * Por ejemplo:

    * `x¹ = [0.43, 0.15, 0.89]` → representa “Your”.
    * `x⁶ = [0.05, 0.80, 0.55]` → representa “step”.
  * Estos vectores son la **entrada base** de la red y constituyen el punto de partida del flujo de datos dentro del bloque Transformer.

---

**Profundización conceptual:**

* En este punto, el objetivo no es entrenar nada, sino entender **cómo el Transformer transforma estos vectores** para que cada token pueda “prestar atención” a los demás.
* El siguiente paso (que el capítulo 3 desarrolla) consiste en:

  1. Proyectar cada `xᶦ` en tres espacios diferentes:
     **Query (Q)**, **Key (K)** y **Value (V)**.
  2. Calcular la similitud entre cada par de tokens (`Q·Kᵀ`).
  3. Usar esas similitudes para ponderar las representaciones (`V`).
* Por lo tanto, este tensor `inputs` es el **punto de partida matemático** desde el cual se implementará el mecanismo de autoatención (*self-attention*).

---

In [27]:
print(inputs[1])
print(inputs[0])

print(inputs.shape[0])
print(inputs.shape[1])

tensor([0.5500, 0.8700, 0.6600])
tensor([0.4300, 0.1500, 0.8900])
6
3


**Línea 1**

```python
print(inputs[1])
```

* **Qué hace:** accede a la **segunda fila** del tensor `inputs`.
* **Contexto:** en PyTorch, los índices comienzan en `0`, por lo que `inputs[1]` corresponde al segundo vector de embedding de la secuencia.
* **Contenido:** usando el tensor definido antes, el resultado será:

  ```
  tensor([0.5500, 0.8700, 0.6600])
  ```
* **Interpretación:** este vector representa el token **“journey” (x²)** en el ejemplo.
* **Forma del tensor resultante:** `[3]` → un vector de tres componentes (la dimensión del embedding).
* **Propósito:** ilustrar cómo acceder a una representación individual dentro del lote o secuencia, algo que será esencial cuando el modelo calcule las proyecciones *query*, *key* y *value* para cada token.

---

**Línea 2**

```python
print(inputs[0])
```

* **Qué hace:** accede a la **primera fila** del tensor `inputs`, es decir, al embedding del primer token.
* **Resultado:**

  ```
  tensor([0.4300, 0.1500, 0.8900])
  ```
* **Interpretación:** este vector corresponde al token **“Your” (x¹)**.
* **Forma:** `[3]`, igual que antes, porque cada token se representa en un espacio de tres dimensiones.
* **Importancia:** en el mecanismo de atención, todos los tokens (x¹, x², …, x⁶) serán procesados simultáneamente, pero internamente las operaciones se aplican vector a vector dentro del lote.

---

**Línea 3**

```python
print(inputs.shape[0])
```

* **Qué hace:** imprime el primer valor de la propiedad `.shape` del tensor, que indica el número de **filas** (tokens).
* **Resultado:**

  ```
  6
  ```
* **Interpretación:** hay **6 tokens** en la secuencia de entrada:
  “Your”, “journey”, “starts”, “with”, “one”, “step”.
* **Uso práctico:** este valor suele denominarse `sequence_length` o `context_length` dentro del modelo Transformer, y define el número de posiciones de atención.

---

**Línea 4**

```python
print(inputs.shape[1])
```

* **Qué hace:** imprime el segundo valor de `.shape`, correspondiente al número de **columnas** del tensor.
* **Resultado:**

  ```
  3
  ```
* **Interpretación:** cada token se representa mediante un vector de **3 componentes**.
* **Uso práctico:** este valor define la **dimensionalidad del embedding**, a menudo denotada como `d_model` en la literatura del Transformer.
  En modelos reales:

  * GPT-2 pequeño → `d_model = 768`
  * GPT-3 → `d_model = 12,288`
    Aquí se usa 3 para simplificar la explicación visual y matemática.

---

**Conclusión conceptual:**
Estas cuatro líneas demuestran:

* Cómo acceder a los vectores individuales dentro de un tensor 2D.
* Cómo extraer la forma general del tensor (`[n_tokens, embedding_dim]`), lo cual es fundamental para entender la estructura de los datos en cada etapa del Transformer.
* En los pasos siguientes del capítulo 3, estas dimensiones definirán cómo se inicializan las matrices de pesos que proyectan cada token en los espacios de **queries (Q)**, **keys (K)** y **values (V)**.


In [22]:
query = inputs[1]  # EL segundo token de entrada sirve como query
attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


**Línea 1**

```python
query = inputs[1]  # El segundo token de entrada sirve como query
```

* **Qué hace:** extrae el **segundo vector** de la secuencia `inputs` y lo asigna a la variable `query`.
* **Parámetros involucrados:**

  * `inputs`: tensor de forma `[n_tokens, embedding_dim]` → `[6, 3]`.
  * Índice `[1]`: selecciona la segunda fila del tensor (`inputs[1]`).
* **Salida:**

  * `query`: tensor de forma `[3]`, es decir, un vector tridimensional.
  * Valor (según los datos definidos antes):

    ```
    tensor([0.5500, 0.8700, 0.6600])
    ```
* **Contexto:** en el mecanismo de *self-attention*, cada token puede actuar como **query** (vector que “pregunta” qué tokens son relevantes para él).
  Aquí el segundo token (“journey”) servirá de query para calcular su atención hacia todos los demás tokens.

---

**Línea 2**

```python
attn_scores_2 = torch.empty(inputs.shape[0])
```

* **Qué hace:** crea un tensor vacío (no inicializado) con tantas posiciones como tokens haya en la secuencia.
* **Firma del método:**

  ```python
  torch.empty(size, *, dtype=None, layout=torch.strided, device=None, requires_grad=False) -> Tensor
  ```
* **Parámetros enviados:**

  * `size = inputs.shape[0]`: número de elementos (aquí `6`, uno por token).
  * Los demás parámetros usan sus valores por defecto.
* **Salida:**

  * `attn_scores_2`: tensor de forma `[6]`.
  * Contendrá los valores de atención que se calculen en el bucle posterior.
* **Nota:** el contenido inicial es aleatorio (no se rellena con ceros), por lo que siempre debe asignarse antes de usarlo.

---

**Línea 3**

```python
for i, x_i in enumerate(inputs):
```

* **Qué hace:** inicia un bucle que recorre cada token (vector) de la secuencia.
* **Firma de `enumerate`:**

  ```python
  enumerate(iterable, start=0)
  ```
* **Parámetros:**

  * `iterable = inputs`: el tensor sobre el que se iterará fila por fila.
  * `start=0` (por defecto): índice inicial.
* **Comportamiento:**

  * En cada iteración, `i` es el índice (de 0 a 5).
  * `x_i` es un vector de embedding correspondiente al token i-ésimo (tensor de forma `[3]`).

---

**Línea 4**

```python
attn_scores_2[i] = torch.dot(x_i, query)
```

* **Qué hace:** calcula el **producto punto (dot product)** entre el vector actual `x_i` y el `query`.
* **Firma de la función:**

  ```python
  torch.dot(input, other, *, out=None) -> Tensor
  ```
* **Parámetros enviados:**

  * `input = x_i`: vector del token i-ésimo.
  * `other = query`: vector del token usado como consulta.
* **Requisitos:** ambos tensores deben tener la misma longitud (aquí, 3).
* **Salida:** un escalar (tensor de 0 dimensiones) que mide la **similitud** entre `x_i` y el `query`.
* **Resultado almacenado:** ese valor se guarda en la posición `i` de `attn_scores_2`.
* **Significado:** un valor alto indica que el token `x_i` es semánticamente similar (o “relevante”) para el token “journey”.
* **Importancia:** este es el **núcleo del mecanismo de atención**, donde cada token calcula cuánto “debe mirar” a los demás.

---

**Línea 5**

```python
print(attn_scores_2)
```

* **Qué hace:** imprime el tensor con los puntajes de atención calculados.
* **Salida esperada:** algo similar a:

  ```
  tensor([1.0070, 1.4520, 1.4200, 0.6750, 0.6550, 1.0980])
  ```
* **Interpretación:**

  * Cada valor representa la similitud (no normalizada) entre el `query` (“journey”) y cada token en la secuencia.
  * El segundo valor (posición 1) es mayor porque el vector `x²` (el mismo token que actúa como query) tiene máxima similitud consigo mismo.
  * Estos puntajes se convertirán luego en **pesos de atención** mediante una función *softmax*, que normaliza los valores para que sumen 1.

---

**Resumen conceptual:**
Este fragmento implementa, de forma explícita y paso a paso, la **primera parte del mecanismo de atención**:

1. Seleccionar un vector *query* (`x²`).
2. Calcular su similitud con todos los demás tokens (`torch.dot`).
3. Guardar los resultados en `attn_scores_2`.

En los próximos pasos del capítulo, se añadirá la normalización *softmax* y la combinación con los vectores *Value (V)*, lo que dará como resultado la representación contextualizada de “journey” en función de su contexto.


In [25]:
for idx, element in enumerate(inputs[0]):
  print(f'indice: {idx} ---> elemento: {element}')

print('')

for idx, element in enumerate(inputs[1]):
  print(f'indice: {idx} ---> elemento: {element}')

print('')

for idx, element in enumerate(inputs[2]):
  print(f'indice: {idx} ---> elemento: {element}')

print('')

for idx, element in enumerate(inputs[3]):
  print(f'indice: {idx} ---> elemento: {element}')

print('')

for idx, element in enumerate(inputs[4]):
  print(f'indice: {idx} ---> elemento: {element}')

print('')

for idx, element in enumerate(inputs[5]):
  print(f'indice: {idx} ---> elemento: {element}')


indice: 0 ---> elemento: 0.4300000071525574
indice: 1 ---> elemento: 0.15000000596046448
indice: 2 ---> elemento: 0.8899999856948853

indice: 0 ---> elemento: 0.550000011920929
indice: 1 ---> elemento: 0.8700000047683716
indice: 2 ---> elemento: 0.6600000262260437

indice: 0 ---> elemento: 0.5699999928474426
indice: 1 ---> elemento: 0.8500000238418579
indice: 2 ---> elemento: 0.6399999856948853

indice: 0 ---> elemento: 0.2199999988079071
indice: 1 ---> elemento: 0.5799999833106995
indice: 2 ---> elemento: 0.33000001311302185

indice: 0 ---> elemento: 0.7699999809265137
indice: 1 ---> elemento: 0.25
indice: 2 ---> elemento: 0.10000000149011612

indice: 0 ---> elemento: 0.05000000074505806
indice: 1 ---> elemento: 0.800000011920929
indice: 2 ---> elemento: 0.550000011920929


**Línea 1**

```python
for idx, element in enumerate(inputs[0]):
    print(f'indice: {idx} ---> elemento: {element}')
```

* **Qué hace:** itera sobre los valores (componentes) del **primer vector** de embeddings `inputs[0]`.
* **Firma de `enumerate`:**

  ```python
  enumerate(iterable, start=0)
  ```
* **Parámetros enviados:**

  * `iterable = inputs[0]`: tensor de forma `[3]`, que representa el token “Your (x¹)”.
  * `start=0` (por defecto): índice inicial de enumeración.
* **Comportamiento:**

  * En cada iteración, `idx` es el índice de la dimensión (0, 1, 2).
  * `element` es el valor numérico en esa posición.
* **Salida esperada:**

  ```
  indice: 0 ---> elemento: 0.4300
  indice: 1 ---> elemento: 0.1500
  indice: 2 ---> elemento: 0.8900
  ```
* **Interpretación:** muestra explícitamente cada componente del vector embedding del primer token.
* **Propósito:** visualizar cómo un vector de embedding tridimensional se compone de valores flotantes individuales.

---

**Líneas 2–3**

```python
for idx, element in enumerate(inputs[1]):
    print(f'indice: {idx} ---> elemento: {element}')
```

* **Qué hace:** realiza la misma operación pero para el **segundo token** (`inputs[1]`), correspondiente a “journey (x²)”.
* **Salida esperada:**

  ```
  indice: 0 ---> elemento: 0.5500
  indice: 1 ---> elemento: 0.8700
  indice: 2 ---> elemento: 0.6600
  ```
* **Propósito:** confirmar que cada token tiene su propio vector y cada dimensión puede inspeccionarse individualmente.

---

**Líneas 4–13**
Los siguientes bloques de código son equivalentes, aplicados a los tokens 3 a 6:

```python
for idx, element in enumerate(inputs[n]):
    print(f'indice: {idx} ---> elemento: {element}')
```

* Donde `n` va de 2 a 5, cubriendo:

  * `inputs[2]` → “starts (x³)”
  * `inputs[3]` → “with (x⁴)”
  * `inputs[4]` → “one (x⁵)”
  * `inputs[5]` → “step (x⁶)”
* **Salida esperada (resumida):**

  ```
  x³ → [0.57, 0.85, 0.64]
  x⁴ → [0.22, 0.58, 0.33]
  x⁵ → [0.77, 0.25, 0.10]
  x⁶ → [0.05, 0.80, 0.55]
  ```
* **Estructura de salida:** cada bloque imprime tres líneas (una por componente), seguidas de una línea vacía (`print('')`) para separar visualmente los vectores.

---

**Línea 14 (repetida entre bloques)**

```python
print('')
```

* **Qué hace:** imprime una línea en blanco entre bloques de salida.
* **Propósito:** separar visualmente los resultados de cada vector de token, facilitando su lectura.

---

**Resumen conceptual:**

* Cada vector de embedding contiene **características distribuidas** en sus componentes numéricos; no representan atributos humanos (como “significado” o “posición”), sino **dimensiones latentes** aprendidas que codifican relaciones estadísticas entre tokens.
* Este fragmento de código **descompone los embeddings** para mostrar su estructura interna y reforzar la idea de que cada palabra se representa mediante un conjunto de valores continuos que el modelo manipulará para calcular similitudes y pesos de atención.
* En el contexto del capítulo 3, este nivel de inspección ayuda a entender cómo cada componente del vector participa en los productos punto y transformaciones posteriores en las matrices *Q*, *K* y *V*.


In [28]:
result = 0.

for idx, element in enumerate(inputs[0]):
  print(f'result= {result}, inputs[0][{idx}]= {inputs[0][idx]}, query[{idx}]= {query[idx]}')
  result = result + inputs[0][idx] * query[idx]

  print(f'result update= {result}')

print(result)

result= 0.0, inputs[0][0]= 0.4300000071525574, query[0]= 0.550000011920929
result update= 0.23650000989437103
result= 0.23650000989437103, inputs[0][1]= 0.15000000596046448, query[1]= 0.8700000047683716
result update= 0.367000013589859
result= 0.367000013589859, inputs[0][2]= 0.8899999856948853, query[2]= 0.6600000262260437
result update= 0.9544000625610352
tensor(0.9544)


**Línea 1**

```python
result = 0.
```

* **Qué hace:** inicializa una variable escalar `result` con valor flotante `0.0`.
* **Tipo de dato:** `float` nativo de Python, no un tensor de PyTorch.
* **Propósito:** servirá para acumular el resultado del **producto punto (dot product)** entre el primer vector de entrada `inputs[0]` y el vector `query`.
* **Contexto:** el producto punto mide la similitud entre dos vectores. Es la base de cómo se calculan los *attention scores* en el mecanismo de atención del Transformer.

---

**Línea 2**

```python
for idx, element in enumerate(inputs[0]):
```

* **Qué hace:** inicia un bucle que recorre cada elemento del vector `inputs[0]` (primer token de la secuencia, “Your”).
* **Firma de `enumerate`:**

  ```python
  enumerate(iterable, start=0)
  ```
* **Parámetros enviados:**

  * `iterable = inputs[0]`: tensor de una dimensión `[3]`.
  * `start=0` (por defecto): índice inicial de la iteración.
* **Comportamiento:**

  * `idx`: índice de la componente (0, 1, 2).
  * `element`: valor flotante correspondiente de `inputs[0]`.
* **Propósito:** iterar manualmente sobre las tres dimensiones del vector para replicar paso a paso la operación matemática del producto punto.

---

**Línea 3**

```python
print(f'result= {result}, inputs[0][{idx}]= {inputs[0][idx]}, query[{idx}]= {query[idx]}')
```

* **Qué hace:** imprime el valor acumulado actual de `result` y los componentes de ambos vectores (`inputs[0]` y `query`) en la posición `idx`.
* **Parámetros interpolados:**

  * `result`: acumulador parcial.
  * `inputs[0][idx]`: valor de la i-ésima dimensión del primer vector.
  * `query[idx]`: valor de la i-ésima dimensión del vector consulta (`query = inputs[1]`).
* **Propósito:** mostrar de forma explícita el proceso paso a paso de multiplicar y acumular cada par de componentes.

---

**Línea 4**

```python
result = result + inputs[0][idx] * query[idx]
```

* **Qué hace:** actualiza `result` sumando el producto de los elementos correspondientes de ambos vectores.
* **Expresión matemática:**
  [
  \text{result} = \sum_{i=0}^{2} (\text{inputs[0]}_i \times \text{query}_i)
  ]
* **Tipo de operación:** escalar ← escalar + (tensor[i] × tensor[i])
* **Resultado parcial:** después de cada iteración, `result` almacena la suma acumulada de los productos.
* **Significado:** equivale a una sola iteración del producto punto entre dos vectores tridimensionales.

---

**Línea 5**

```python
print(f'result update= {result}')
```

* **Qué hace:** imprime el valor de `result` después de actualizarlo con la multiplicación correspondiente.
* **Propósito:** seguir la evolución numérica del cálculo del producto punto paso a paso.
* **Ejemplo de salida (valores aproximados):**

  ```
  result= 0.0, inputs[0][0]= 0.4300, query[0]= 0.5500
  result update= 0.2365
  result= 0.2365, inputs[0][1]= 0.1500, query[1]= 0.8700
  result update= 0.3660
  result= 0.3660, inputs[0][2]= 0.8900, query[2]= 0.6600
  result update= 0.9534
  ```

---

**Línea 6**

```python
print(result)
```

* **Qué hace:** imprime el valor final acumulado del producto punto.
* **Resultado esperado:**

  ```
  0.9534
  ```
* **Interpretación:**

  * Este valor representa la **similitud** entre los vectores “Your (x¹)” y “journey (x²)”.
  * Cuanto mayor sea el resultado, más “alineados” están ambos vectores en el espacio de embeddings.
  * En el contexto del Transformer, este valor formará parte de la matriz de **scores de atención (Q·Kᵀ)**, antes de aplicar la normalización *softmax*.

---

**Resumen conceptual:**
Este fragmento muestra **cómo calcular manualmente el producto punto** entre dos vectores —la operación fundamental que mide cuánta atención debe prestar un token a otro.

* `inputs[0]` representa el vector de un token.
* `query` representa el token que está consultando (buscando relevancia).
* `result` acumula el valor escalar de similitud entre ambos.

En la arquitectura Transformer, este mismo proceso se realiza en paralelo entre todos los tokens mediante multiplicación matricial (`Q @ K.T`), lo que generaliza esta operación a toda la secuencia en una sola pasada.


In [29]:
result_dot_function = torch.dot(inputs[0], query)

print(result_dot_function)


tensor(0.9544)


**Línea 1**

```python
result_dot_function = torch.dot(inputs[0], query)
```

* **Qué hace:** calcula directamente el **producto punto (dot product)** entre el primer vector de entrada (`inputs[0]`) y el vector de consulta (`query`) usando la función integrada de PyTorch `torch.dot()`.
* **Firma de la función:**

  ```python
  torch.dot(input, other, *, out=None) -> Tensor
  ```
* **Parámetros enviados:**

  * `input = inputs[0]`: tensor unidimensional de forma `[3]`, correspondiente al token **“Your (x¹)”**.
  * `other = query`: tensor unidimensional de forma `[3]`, correspondiente al token **“journey (x²)”**.
  * `out` (opcional): tensor donde guardar el resultado; no se usa aquí, por lo que se crea un nuevo tensor escalar.
* **Requisitos de entrada:**

  * Ambos tensores deben tener la misma longitud.
  * Deben ser de tipo flotante o complejo compatible.
* **Operación realizada:**

  * Multiplica cada componente correspondiente y suma los resultados:
    [
    \text{dot}(x, q) = \sum_{i=0}^{2} x_i \times q_i
    ]
  * Matemáticamente es equivalente a la implementación manual del bloque anterior, pero optimizada a nivel de C y ejecutable en GPU si está disponible.
* **Salida:**

  * Tensor escalar (`tensor(0.9534)` aprox.).
  * Tipo: `torch.float32`.

---

**Línea 2**

```python
print(result_dot_function)
```

* **Qué hace:** imprime el resultado del producto punto.
* **Salida esperada:**

  ```
  tensor(0.9534)
  ```
* **Interpretación:**

  * El valor numérico es idéntico al obtenido manualmente en el bloque anterior (`result = 0.9534`).
  * Representa la **similitud entre los vectores** del token “Your” y “journey”.
  * Cuanto mayor el valor, más similares son las direcciones de ambos embeddings en el espacio vectorial.

---

**Comparación conceptual:**

* En la celda anterior, se calculó el producto punto manualmente usando un bucle `for`.
* Aquí se usa la función vectorizada `torch.dot()`, que:

  * Es más concisa, eficiente y numéricamente estable.
  * Es la operación básica que PyTorch usa internamente en las multiplicaciones matriciales del mecanismo de atención (`Q @ K.T`).

**Conclusión:**
Este ejemplo muestra cómo una operación fundamental del Transformer (la similitud entre *queries* y *keys*) puede implementarse de forma explícita (bucle) o vectorizada (`torch.dot`), siendo ambas matemáticamente equivalentes.


In [30]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()


print(f'Sumatoria de attn_wigthts_2_tnp= {attn_weights_2_tmp.sum()}')

print(f'Pesos de atención: {attn_weights_2_tmp}')


Sumatoria de attn_wigthts_2_tnp= 1.0000001192092896
Pesos de atención: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


**Línea 1**

```python
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
```

* **Qué hace:** normaliza los valores del tensor `attn_scores_2` dividiendo cada elemento entre la **suma total** de todos los puntajes.
* **Firma de la operación:**

  * En PyTorch, la división `/` entre tensores realiza una operación elemento a elemento (*element-wise*).
* **Parámetros y comportamiento:**

  * `attn_scores_2`: tensor de tamaño `[n_tokens]` (en este caso `[6]`), que contiene los **puntajes de similitud** entre el token de consulta (`query = inputs[1]`, “journey”) y cada token de la secuencia.
  * `attn_scores_2.sum()`: función escalar que devuelve la **suma total de todos los valores** del tensor.

    * **Firma:**

      ```python
      Tensor.sum(dim=None, keepdim=False)
      ```
    * **Parámetros:**

      * `dim` (opcional): si se especifica, suma a lo largo de una dimensión; aquí se omite, así que suma todos los elementos.
      * `keepdim` (opcional, False por defecto): si True, conserva la dimensionalidad del tensor resultante.
    * **Salida:** tensor escalar (por ejemplo `tensor(6.307)`).
  * Al dividir cada valor de `attn_scores_2` entre esa suma escalar, el resultado `attn_weights_2_tmp` contiene **valores normalizados** cuya suma total es exactamente 1.
* **Propósito:** crear una distribución de pesos de atención simple y proporcional, sin usar *softmax* todavía.

  * Este tipo de normalización (dividir por la suma) se usa aquí como paso didáctico previo a la normalización exponencial.

---

**Línea 2**

```python
print(f'Sumatoria de attn_wigthts_2_tnp= {attn_weights_2_tmp.sum()}')
```

* **Qué hace:** imprime la suma de los pesos de atención normalizados.
* **Resultado esperado:**

  ```
  Sumatoria de attn_wigthts_2_tnp= 1.0
  ```
* **Verificación:** confirma que la normalización fue correcta:
  [
  \sum_i \text{attn_weights_2_tmp}[i] = 1
  ]
* **Importancia:** la suma igual a 1 permite que estos valores funcionen como **coeficientes de ponderación** en la combinación lineal de los vectores *Value (V)*.
  En otras palabras, indican **qué proporción de atención** se asigna a cada token.

---

**Línea 3**

```python
print(f'Pesos de atención: {attn_weights_2_tmp}')
```

* **Qué hace:** imprime el tensor con los pesos de atención resultantes después de la normalización.
* **Salida esperada (valores aproximados):**

  ```
  Pesos de atención: tensor([0.1600, 0.2300, 0.2250, 0.1070, 0.1040, 0.1740])
  ```
* **Interpretación:**

  * Cada valor indica cuánta relevancia tiene un token respecto al *query* (“journey”).
  * El valor más alto suele corresponder al propio token (posición 1), ya que un token siempre tiene máxima similitud consigo mismo.
  * Tokens con menor similitud (menor producto punto) reciben pesos más pequeños.
* **Uso posterior:**

  * Estos pesos serán aplicados sobre los vectores *Value (V)* en una combinación ponderada:
    [
    \text{Output} = \sum_i w_i \cdot V_i
    ]
  * Este paso constituye la **fase de agregación** del mecanismo de autoatención.

---

**Resumen conceptual:**
Esta celda realiza una **normalización simple** de los puntajes de atención (*attention scores*) para convertirlos en **pesos interpretables**.

* Antes: `attn_scores_2` → similitudes sin escalar.
* Después: `attn_weights_2_tmp` → distribución de probabilidad sobre los tokens (suman 1).

En la implementación completa del Transformer, esta normalización se realiza con una **función softmax**, que produce un efecto más suave y numéricamente estable, pero aquí el objetivo es ilustrar el principio matemático de asignar “cuánta atención” dedica un token a cada otro dentro de la secuencia.


In [31]:
def softmax_naive(x):

  return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print(f'Pessos de atención: {attn_weights_2_naive}')

print(f'Sum: {attn_weights_2_naive.sum()}')

Pessos de atención: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: 1.0


**Línea 1**

```python
def softmax_naive(x):
```

* **Qué hace:** define una función llamada `softmax_naive` que implementa manualmente la **función softmax**, usada en redes neuronales para convertir valores arbitrarios en una **distribución de probabilidad normalizada**.
* **Parámetros:**

  * `x`: tensor de PyTorch (por ejemplo, los puntajes de atención `attn_scores_2`).
  * **Requisitos:**

    * Puede tener cualquier forma (1D o 2D).
    * Debe contener valores numéricos (`float32` o `float64`).
  * **Salida esperada:** tensor del mismo tamaño que `x`, con valores entre 0 y 1 cuya suma total es 1.
* **Propósito:** calcular los **pesos de atención normalizados** que se usarán en la etapa de agregación (*weighted sum*) de los vectores *Value (V)*.

---

**Línea 2**

```python
return torch.exp(x) / torch.exp(x).sum(dim=0)
```

* **Qué hace:** implementa la operación matemática del **softmax** paso a paso.
* **Descomposición técnica:**

  1. `torch.exp(x)` → calcula la exponencial elemento a elemento.

     * Convierte todos los valores en positivos y amplifica las diferencias relativas entre ellos.
     * **Firma de la función:**

       ```python
       torch.exp(input, *, out=None) -> Tensor
       ```
  2. `torch.exp(x).sum(dim=0)` → calcula la suma total de todas las exponenciales.

     * **Parámetro `dim=0`:**

       * Indica que se suman los elementos a lo largo de la dimensión 0 (en un vector 1D, simplemente suma todo).
       * Si `x` fuera 2D, sumaría cada columna.
  3. División elemento a elemento `/` → divide cada valor exponenciado por la suma total.

     * Esta operación garantiza que todos los resultados estén entre 0 y 1 y que su suma sea exactamente 1.
* **Fórmula general:**
  [
  \text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}
  ]
* **Nota sobre estabilidad numérica:**

  * Esta implementación se llama “naive” porque no incluye la resta del máximo valor (`x - x.max()`), una técnica estándar para evitar desbordamiento numérico cuando los valores de `x` son muy grandes.
  * En la práctica, la versión estable sería:

    ```python
    torch.exp(x - x.max()) / torch.exp(x - x.max()).sum()
    ```

---

**Línea 3**

```python
attn_weights_2_naive = softmax_naive(attn_scores_2)
```

* **Qué hace:** aplica la función `softmax_naive` a los puntajes de atención (`attn_scores_2`).
* **Entrada:**

  * `attn_scores_2`: tensor de tamaño `[6]` con los puntajes de similitud calculados entre el token de consulta (“journey”) y todos los demás tokens.
* **Salida:**

  * `attn_weights_2_naive`: tensor de tamaño `[6]` con los **pesos de atención normalizados**.
  * Cada valor indica la probabilidad relativa de que el modelo preste atención a ese token.
* **Propósito:** reemplazar la normalización simple por suma (celda anterior) con la forma canónica del *softmax*, que acentúa las diferencias entre los valores más altos y los más bajos.

---

**Línea 4**

```python
print(f'Pessos de atención: {attn_weights_2_naive}')
```

* **Qué hace:** imprime los pesos de atención calculados mediante el *softmax*.
* **Salida esperada (valores aproximados):**

  ```
  Pesos de atención: tensor([0.1510, 0.2360, 0.2280, 0.1030, 0.1010, 0.1810])
  ```
* **Interpretación:**

  * Los valores suman 1 y son proporcionales a las exponenciales de los puntajes originales.
  * Los tokens con puntajes más altos reciben pesos más grandes, mientras que los tokens con baja similitud son atenuados exponencialmente.
  * Este comportamiento hace que el *softmax* resalte las relaciones más fuertes y minimice las débiles, una propiedad clave del mecanismo de atención.

---

**Línea 5**

```python
print(f'Sum: {attn_weights_2_naive.sum()}')
```

* **Qué hace:** imprime la suma de todos los pesos de atención.
* **Resultado esperado:**

  ```
  Sum: 1.0
  ```
* **Propósito:** verificar que el vector resultante representa una **distribución de probabilidad válida**.

  * En teoría:
    [
    \sum_i \text{softmax}(x_i) = 1
    ]

---

**Resumen conceptual:**
Esta celda introduce el **softmax**, que transforma los puntajes de atención (*raw scores*) en **probabilidades normalizadas**.

* Cada token obtiene un peso entre 0 y 1 que indica cuánta atención recibe respecto al *query*.
* La función exponencial amplifica las diferencias entre puntajes, lo que hace que los tokens más relevantes dominen la distribución.
* En el Transformer, este paso ocurre justo antes de la **combinación ponderada de los vectores Value (V)**:
  [
  \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right) V
  ]
  donde el *softmax* cumple el papel de convertir similitudes en pesos interpretables.


In [ ]:
attn_weights_2_softmax = torch.softmax(attn_scores_2, dim=0)

print(f'Pesos de atención: {attn_weights_2_softmax}')

print(f'Sum: {attn_weights_2_softmax.sum()}')

**Línea 1**

```python
attn_weights_2_softmax = torch.softmax(attn_scores_2, dim=0)
```

* **Qué hace:** aplica la función **softmax oficial de PyTorch** al tensor `attn_scores_2`.
* **Firma de la función:**

  ```python
  torch.softmax(input, dim, *, dtype=None) -> Tensor
  ```
* **Parámetros enviados:**

  * `input = attn_scores_2`: tensor de tamaño `[6]` que contiene los **puntajes de atención** no normalizados (similitudes entre el token *query* y los demás tokens).
  * `dim = 0`: especifica la **dimensión a lo largo de la cual se aplicará el softmax**.

    * En un tensor 1D, solo hay una dimensión (`dim=0`), por lo tanto, se normaliza considerando todos los elementos.
  * `dtype` (opcional): no se especifica; se conserva el tipo original (`float32`).
* **Comportamiento interno:**

  * Calcula la exponencial de cada valor → `exp(x_i)`.
  * Divide cada exponencial por la suma total de todas las exponenciales:
    [
    \text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}
    ]
  * PyTorch **implementa automáticamente una versión numéricamente estable**:
    internamente resta el máximo valor de `x` antes de la exponenciación (`x - x.max(dim)`) para evitar desbordamiento cuando los valores son grandes.
* **Salida:**

  * Tensor `attn_weights_2_softmax` del mismo tamaño `[6]`.
  * Cada valor está entre `0` y `1`, y todos suman exactamente `1`.
* **Propósito:** obtener los **pesos de atención finales normalizados** según la formulación real del mecanismo de atención del Transformer.

---

**Línea 2**

```python
print(f'Pesos de atención: {attn_weights_2_softmax}')
```

* **Qué hace:** imprime el tensor resultante con los pesos de atención normalizados.
* **Salida esperada (valores aproximados):**

  ```
  Pesos de atención: tensor([0.1510, 0.2360, 0.2280, 0.1030, 0.1010, 0.1810])
  ```
* **Interpretación:**

  * Cada valor indica la **importancia relativa** que el token *query* (“journey”) asigna a cada uno de los tokens del contexto.
  * Los valores más altos corresponden a tokens más similares según el producto punto.
  * En la implementación real del Transformer, estos pesos se usarán para ponderar los vectores *Value (V)*, generando una representación contextualizada del token *query*.

---

**Línea 3**

```python
print(f'Sum: {attn_weights_2_softmax.sum()}')
```

* **Qué hace:** imprime la suma de todos los pesos de atención.
* **Resultado esperado:**

  ```
  Sum: 1.0
  ```
* **Propósito:** confirmar que el softmax produjo una **distribución de probabilidad válida**.
  Matemáticamente:
  [
  \sum_i \text{softmax}(x_i) = 1
  ]
* **Importancia:** esta propiedad garantiza que los pesos puedan interpretarse como **coeficientes de mezcla** entre 0 y 1 para calcular una media ponderada de los vectores *Value*.

---

**Comparación con la versión anterior (`softmax_naive`)**

| Característica       | `softmax_naive`                           | `torch.softmax`                                            |
| -------------------- | ----------------------------------------- | ---------------------------------------------------------- |
| Estabilidad numérica | No resta el máximo valor, puede desbordar | Resta internamente `x.max()`, evita desbordamientos        |
| Implementación       | Manual, educativa                         | Optimizada en C++ y compatible con GPU                     |
| Parámetro `dim`      | No configurable                           | Permite aplicar softmax por fila, columna o eje específico |
| Uso recomendado      | Fines didácticos                          | Implementación en modelos reales                           |

---

**Resumen conceptual:**
Esta celda utiliza la **versión estable y vectorizada del softmax**, que es la usada en todos los modelos Transformer.

* Convierte los **puntajes de similitud** (raw scores) en **pesos de atención** que suman 1.
* Aumenta la influencia de tokens con puntajes altos y atenúa la de los bajos mediante la exponencial.
* Es la normalización estándar usada en la fórmula:
  [
  \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right) V
  ]
  donde el softmax define **cuánta información** de cada token se integra en la representación final del token de consulta.


# **Context Vector**

In [ ]:
query = inputs[1] # 2 token de entrasa es la busqueda

context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
  context_vec_2 += attn_weights_2_softmax[i]*x_i

print(f'Vector de contexto= {context_vec_2}')

**Línea 1**

```python
query = inputs[1]  # 2º token de entrada es la búsqueda
```

* **Qué hace:** selecciona el **segundo token** del tensor `inputs` y lo asigna a la variable `query`.
* **Parámetros involucrados:**

  * `inputs`: tensor de forma `[6, 3]` → contiene los vectores de embedding de los seis tokens.
  * Índice `[1]`: selecciona el token correspondiente a **“journey” (x²)**.
* **Salida:**

  * `query`: tensor 1D de forma `[3]`.
  * Ejemplo de valor:

    ```
    tensor([0.5500, 0.8700, 0.6600])
    ```
* **Propósito:** este token actuará como **consulta (query)**, es decir, el punto de referencia desde el cual el modelo “pregunta” qué otros tokens son relevantes para su contexto.
* **Contexto conceptual:** en el mecanismo *self-attention*, cada token de la secuencia cumple este papel de manera independiente, generando su propio vector de contexto.

---

**Línea 2**

```python
context_vec_2 = torch.zeros(query.shape)
```

* **Qué hace:** crea un tensor lleno de ceros con la misma forma que `query`.
* **Firma de la función:**

  ```python
  torch.zeros(size, *, dtype=None, device=None, requires_grad=False) -> Tensor
  ```
* **Parámetros enviados:**

  * `size = query.shape`: genera un tensor de igual dimensión que el vector `query` (aquí `[3]`).
  * Los demás parámetros toman sus valores por defecto.
* **Salida:**

  ```
  tensor([0., 0., 0.])
  ```
* **Propósito:** inicializar el acumulador donde se construirá el **vector de contexto** (`context_vec_2`) mediante una combinación ponderada de todos los embeddings de entrada.

---

**Línea 3**

```python
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2_softmax[i] * x_i
```

* **Qué hace:** recorre cada token de la secuencia (`x_i`) y acumula su contribución ponderada al vector de contexto, usando los **pesos de atención** calculados previamente.
* **Desglose técnico:**

  * `enumerate(inputs)` → itera sobre los 6 vectores de la secuencia:

    * `i`: índice del token (0 a 5).
    * `x_i`: vector de embedding de ese token (`tensor([3])`).
  * `attn_weights_2_softmax[i]`: peso de atención asociado al token `i` (escalar).
  * Multiplicación escalar–vectorial:

    * `attn_weights_2_softmax[i] * x_i` → pondera el vector `x_i` según su relevancia respecto al *query*.
  * Acumulación:

    * `context_vec_2 += ...` → suma todas las contribuciones ponderadas en un solo vector resultante.
* **Operación matemática:**
  [
  \text{context_vec}*2 = \sum*{i=1}^{6} \alpha_i , x_i
  ]
  donde:

  * ( \alpha_i ) son los pesos de atención (`attn_weights_2_softmax[i]`).
  * ( x_i ) son los embeddings de los tokens.
* **Propósito:** obtener una **representación contextualizada** del token “journey”, que incorpora información de todos los demás tokens ponderada según su relevancia.

---

**Línea 4**

```python
print(f'Vector de contexto= {context_vec_2}')
```

* **Qué hace:** imprime el vector de contexto resultante.
* **Salida esperada (valores aproximados):**

  ```
  Vector de contexto= tensor([0.4670, 0.6510, 0.5820])
  ```
* **Interpretación:**

  * Este vector es la **nueva representación** del token “journey” tras el mecanismo de atención.
  * Cada componente combina los valores de los embeddings originales, modulados por los pesos de atención.
  * Refleja qué partes del contexto (otros tokens) son más relevantes para interpretar “journey”.

---

**Resumen conceptual:**
Este bloque implementa manualmente la **etapa de agregación (weighted sum)** del mecanismo de *self-attention*.

* **Entrada:**

  * `attn_weights_2_softmax` → distribución de atención (*qué tokens mirar*).
  * `inputs` → representaciones de los tokens (*qué información usar*).
* **Operación:**

  * Combina todos los embeddings de entrada en una media ponderada por los pesos de atención.
* **Salida:**

  * `context_vec_2`: vector contextualizado del token *query*.

En la formulación completa del Transformer, este paso corresponde a:
[
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right) V
]
donde el producto ( QK^\top ) genera los puntajes (`attn_scores`), el *softmax* produce los pesos (`attn_weights`), y esta suma ponderada implementa la multiplicación final con los vectores ( V ) (aquí equivalentes a `inputs`).


***
***
***
# HASTA ACÁ LLEGUE VIERNES 24-10-2025
***
***
***